In [1]:

from isv_nlp_utils import constants
from isv_nlp_utils.slovnik import get_slovnik
# from isv_translate import translate_sentence, postprocess_translation_details, prepare_parsing

from ast import literal_eval
import os
import glob


In [2]:
from isv_nlp_utils.flavorization.tokenizer import extract_stem_prefix_suffix

In [3]:
slovnik = get_slovnik()
slovnik = slovnik['words']

morph = constants.create_etm_analyzer(r"C:\dev\ISV_data_gathering\\")


Found 'slovnik.pkl' file, using it


In [4]:

isv_dict = morph._units[0][0].dict


In [6]:
import pickle

with open("partial_verb_prefixes_and_nouns.pkl", "rb") as f:
    partial_verb_prefixes = pickle.load(f)
type(partial_verb_prefixes)

pandas.core.frame.DataFrame

In [11]:
partial_verb_prefixes

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns
5,11,abdikovati,NaN,v.intr. ipf.,abdicate,I,verb,?,?,abdikovati,abdikovati,abdik,abdikovati,|abdikacija
16,6119,abonovati,NaN,v.tr. ipf.,"subscribe, engage",F,verb,?,?,abonovati,abonovati,abon,abonovati,
28,19625,absorbovati,NaN,v.tr. ipf.,absorb,I,verb,?,?,absorbovati,absorbovati,absorb,absorbovati,|absorbcija
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,abstract,I,verb,?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,
43,24070,adaptovati,NaN,v.tr. ipf./pf.,adapt,I,verb,?,?,adaptovati,adaptovati,adapt,adaptovati,|adaptacija
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473,žužati,NaN,v.intr. ipf.,buzz,NaN,verb,?,?,žužati,žužati,žuž,žužati,
17913,-37005,råzpečętati,NaN,v.tr. pf.,unseal,NaN,verb,råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,
17914,-37006,råzpečętyvati,NaN,v.tr. ipf.,unseal,NaN,verb,råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,
17917,-30731,smogti,NaN,v.tr. pf.,"manage to, be able to, cope, make it",NaN,verb,?,?,mogti,smogti,smogti,mogti,


In [47]:
partial_verb_prefixes['base_noun'] = ""

In [14]:
def write_base_verb(indices, base_verb):
    partial_verb_prefixes.loc[
        indices,
        'base_verb'
    ] = base_verb


## архив

In [29]:
simplified_verbs = partial_verb_prefixes.query(
    "partOfSpeech.str.contains('refl.') and ',' not in isv and not isv.str.endswith('ti sę')"
).isv.apply(lambda x: x.replace(" sobě", "").partition(' sę')[0])

partial_verb_prefixes.loc[simplified_verbs.index, 'isv'] = simplified_verbs

In [30]:
simplified_verbs2 = partial_verb_prefixes.query(
    "partOfSpeech.str.contains('refl.') and ',' not in isv and isv.str.endswith('ti sę')"
).isv.apply(lambda x: x.replace(" sobě", "").partition(' sę')[0])

partial_verb_prefixes.loc[simplified_verbs2.index, 'isv'] = simplified_verbs2

In [69]:
partial_verb_prefixes['left_stem_cand'] = partial_verb_prefixes.isv.apply(lambda x: bite_all_suffixes_off(x, '')[0])

partial_verb_prefixes['right_stem_cand'] = partial_verb_prefixes.isv.apply(lambda x: 
   bite_all_prefixes_off(x, '')[0] if morph.word_is_known(bite_all_prefixes_off(x, '')[0]) else "?" + bite_all_prefixes_off(x, '')[0] + "?"
)
partial_verb_prefixes

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand
5,11,abdikovati,NaN,v.intr. ipf.,1,abdicate,NaN,I,отрекаться,адракацца,...,{abdikovať},"{откажува, абдицира}",{отрекаться},{абдикирам},?,?,,abdikovati,abdik,abdikovati
16,6119,abonovati,NaN,v.tr. ipf.,1,"subscribe, engage",NaN,F,"подписываться, абонировать","падпісвацца, абаніраваць",...,"{zapísať sa (na odber), prihlásiť sa, predplat...","{претплатува, абонира}","{подписываться, абонировать}",{абонирам},?,?,,abonovati,abon,abonovati
28,19625,absorbovati,NaN,v.tr. ipf.,2,absorb,NaN,I,"абсорбировать, поглощать","абсарбаваць, паглынаць",...,"{absorbovať, vstrebávať}","{впива, апсорбира}","{абсорбировать, поглощать}",{абсорбирам},?,?,,absorbovati,absorb,absorbovati
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,1,abstract,NaN,I,абстрагировать,абстрагаваць,...,{abstrahovať},"{издвојува, апстрахира}",{абстрагировать},{абстрахирам се от},?,?,,abstrahovati,abstrah,abstrahovati
43,24070,adaptovati,NaN,v.tr. ipf./pf.,1,adapt,NaN,I,адаптировать,"адаптаваць, прыстасаваць",...,"{adaptovať, prispôsobiť}","{адаптира, прилагодува}",{адаптировать},"{пригаждам, адаптирам}",?,?,,adaptovati,adapt,adaptovati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473,žužati,NaN,v.intr. ipf.,1,buzz,NaN,NaN,жужжать,"гусці, гудзець",...,{bzučať},{зуи},{жужжать},{жужа},?,?,,žužati,žuž,žužati
17913,-37005,råzpečętati,NaN,v.tr. pf.,NaN,unseal,NaN,NaN,распечатать (открыть),распячатаць (адчыніць),...,"{rozpečatiť, odpečatiť}","{распечати, отпечати}",{распечатать (открыть)},"{отпечатам (отворя), !разпечатам}",råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati
17914,-37006,råzpečętyvati,NaN,v.tr. ipf.,NaN,unseal,NaN,NaN,распечатывать (открывать),распячатваць (адчыніць),...,"{rozpečaťovať, odpečaťovať}","{отпечатува, распечатува}",{распечатывать (открывать)},"{отпечатвам (отварям), !разпечатвам}",råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?
17917,-30731,smogti,NaN,v.tr. pf.,NaN,"manage to, be able to, cope, make it",NaN,NaN,"смочь, суметь","змагчы, здолець",...,"{zvládnuť, zmôcť}",{!смогне},"{смочь, суметь}",{!смогвам},?,?,,smogti,smogti,mogti


In [51]:
anomalies = []
i = 0

cnt = Counter()
cnt2 = Counter()
mbs = []

for k, g in partial_verb_prefixes.groupby('left_stem_cand'):
    if g.base_verb.apply(len).min() > 0:
        continue
    cnt[len(g.isv.unique())] += 1
    if g.base_verb.apply(len).max() > 0:
        cnt2[len(g.isv.unique())] += 1
        mbs.append((k, g.isv_orig.values, g.base_verb.values))

    if len(k) > 1 and len(g.isv.unique()) == 2:
        i += 1
        aspect_stats = {}
        for aspect in ['ipf.', 'pf.']:
            # print(aspect, len(g[g.partOfSpeech.str.contains(" " + aspect)]))
            aspect_stats[aspect] = list(g[g.partOfSpeech.str.contains("[ .]" + aspect)].isv.values)
        if any(len(x) == 0 for x in aspect_stats.values()):
            anomalies.append((aspect_stats, g.partOfSpeech.values.tolist(), g.en.values.tolist(), g.ru.values.tolist(), g.pl.values.tolist()))
            #print(aspect_stats)
            #print(g.en.values.tolist())
            #print(g.ru.values.tolist())
        else:
            bited, prefices = zip(
                *g.isv.apply(lambda x: bite_all_prefixes_off(x, "")).values.tolist()
            )
            if set(prefices) == {'’'}:
                base_verb = g.loc[g.isv.apply(len).idxmin()].isv
                print(g.isv.values.tolist(), "->", base_verb)
                print(g.en.unique())
                print()
                write_base_verb(g.index, base_verb)
            else:
                stems = g.isv.apply(
                    lambda x: bite_all_prefixes_off(x, "")[0]
                    # lambda x: x[len(bite_all_prefixes_off(x, "").replace('’', '')):]
                )
                for verb_cand in sorted(stems.values, key=len):
                    if morph.word_is_known(verb_cand):
                        if {verb_cand} == {v.normal_form for v in morph.parse(verb_cand)}:
                            base_verb_idx = partial_verb_prefixes.query("isv == @verb_cand").index[0]
                            add_index = g.index.tolist() + [base_verb_idx]
                            print(g.isv.values.tolist(), "+", verb_cand, "->", verb_cand)
                            print(g.en.unique())
                            print(partial_verb_prefixes.loc[base_verb_idx, 'en'])
                            print()
                            write_base_verb(add_index, verb_cand)
                            break
                else:
                    base_verb = g.loc[g.isv.apply(len).idxmin()].isv
                    print(g.isv.values.tolist(), "->", "???", base_verb)
                    write_base_verb(g.index, base_verb)
                
len(anomalies)

['obriti', 'obriti', 'obryvati'] -> ??? obriti
['obrušati', 'obrušiti'] + rušiti -> rušiti
['collapse, give way']
knock down

['obråbotati', 'obråbotyvati'] + råbotati -> råbotati
['process']
work

['obrěmeniti', 'obrěmenjati'] -> ??? obrěmeniti
['obstųpati', 'obstųpiti'] -> ??? obstųpati
['obsvětliti', 'obsvětljati'] -> ??? obsvětliti
['obsypati', 'obsypyvati'] -> ??? obsypati
['obsęgati', 'obsęgnųti'] -> ??? obsęgati
['obsějati', 'obsějivati'] -> ??? obsějati
['obtęžati', 'obtęžiti'] -> ??? obtęžati
['obuzdati', 'obuzdyvati'] -> ??? obuzdati
['obučati', 'obučiti'] -> ??? obučati
['obviti', 'obvivati'] -> ??? obviti
['obvažati', 'obvažiti'] -> ??? obvažati
['obvęzati', 'obvęzati', 'obvęzyvati', 'obvęzyvati'] -> ??? obvęzati
['oběćati', 'oběćivati'] -> ??? oběćati
['ocěniti', 'ocěnjati'] + cěniti -> cěniti
['appreciate, rate, value, assess, estimate, evaluate']
appreciate

['oddaliti', 'oddaliti', 'oddaljati', 'oddaljati'] -> ??? oddaliti
['odděliti', 'odděljati'] + děliti -> děliti
['

['pohristijaniti', 'pohristijanjati'] -> ??? pohristijaniti
['poiskati', 'poiskyvati'] + iskati -> iskati
['search, seek, look for' 'search, look up']
search, seek, look for

['pojaviti', 'pojavjati'] + javiti -> javiti
['appear, show up']
appear, become visible, come in sight

['pokarati', 'pokarati', 'pokarjati'] + karati -> karati
['punish' 'scold, rebuke, reprimand, reproach' 'subdue, subjugate']
punish

['pokazati', 'pokazyvati'] + kazati -> kazati
['show']
seem

['politi', 'polivati'] + liti -> liti
['water']
pour

['polučati', 'polučiti'] + lučiti -> lučiti
['obtain']
beam

['popiti', 'popivati'] + piti -> piti
['sip']
drink

['poplaviti', 'poplavjati'] -> ??? poplaviti
['popraviti', 'popravjati'] + praviti -> praviti
['correct, repair, mend, improve, rectify']
edit

['poråbiti', 'poråbovati'] + råbiti -> råbiti
['enslave']
do, make

['porųčati', 'porųčiti'] + rųčiti -> rųčiti
['guarantee, warrant, vouch']
guarantee, warrant, vouch

['posrěbriti', 'posrěbrjati'] -> ??? posrěbrit

['råzkydati', 'råzkydyvati'] + kydati -> kydati
['throw around, scatter']
throw

['råzliti', 'råzlivati'] + liti -> liti
['spill']
pour

['råzličati', 'råzličati', 'råzličiti'] -> ??? råzličati
['råzlųčati', 'råzlųčiti'] -> ??? råzlųčati
['råzmazati', 'råzmazyvati'] + mazati -> mazati
['spread, smear, smudge']
smear, besmear, stain, soil

['råzmetati', 'råzmetyvati'] -> ??? råzmetati
['råzmokati', 'råzmoknųti'] + moknųti -> moknųti
['get soaked, go soggy']
get wet

['råzmysliti', 'råzmysljati'] + mysliti -> mysliti
['contemplate, ponder, wonder, meditate']
think

['råzmękčati', 'råzmękčiti'] + mękčiti -> mękčiti
['soften up, mollify']
soften

['råzměniti', 'råzměnjati'] + měnjati -> měnjati
['exchange']
alternate, vary, fluctuate

['råzměriti', 'råzměrjati'] + měriti -> měriti
['measure off']
measure

['råzměšati', 'råzměšati', 'råzměšivati', 'råzměšivati'] + měšati -> měšati
['mix up' 'puzzle']
mix, blend

['råzorųžati', 'råzorųžiti'] -> ??? råzorųžati
['råzočarovati', 'råzočarovyvati

['utěšati', 'utěšiti'] + těšiti -> těšiti
['console, comfort']
enjoy, take pleasure in

['uvažati', 'uvažati', 'uvažiti'] -> ??? uvažati
['uveličati', 'uveličiti'] -> ??? uveličati
['uvędati', 'uvędnųti'] -> ??? uvędati
['uvěriti', 'uvěriti', 'uvěriti', 'uvěrjati', 'uvěrjati'] -> ??? uvěriti
['učarovati', 'učarovyvati'] + čarovati -> čarovati
['cast a spell']
perform magic

['užiti', 'uživati'] + žiti -> žiti
['use, enjoy']
live

['užasati', 'užasnųti'] -> ??? užasati
['vględati', 'vględěti'] + ględěti -> ględěti
['peer, gaze, stare, look carefully']
look, watch, look at

['vključati', 'vključati', 'vključiti', 'vključiti'] -> ??? vključati
['vměšati', 'vměšivati', 'vměšivati'] + měšati -> měšati
['interfere, intervene' 'involve (somebody in a project), mix into']
mix, blend

['vnikati', 'vniknųti'] -> ??? vnikati
['vpiti', 'vpivati'] + piti -> piti
['absorb']
drink

['vpisati', 'vpisyvati'] + pisati -> pisati
['enter (data)']
write

['vrųbati', 'vrųbyvati'] + rųbati -> rųbati
['#carve

64

In [90]:
r, p = bite_all_prefixes_off(x, '')
p2 = p.replace('’', "")
print(x, p, r, morph.word_is_known(r), p2)
for len_cand in range(1, len(p2)):
    word_cand = p2[-len_cand:] + r
    print(len_cand, word_cand, morph.word_is_known(word_cand))
    if morph.word_is_known(word_cand):
        n += 1
        print(len_cand, word_cand, morph.word_is_known(word_cand))


prěprogramovati prě’pro’ gramovati False prěpro
1 ogramovati False
2 rogramovati False
3 programovati True
3 programovati True
4 ěprogramovati False
5 rěprogramovati False


In [95]:
n = 0

for i, row in partial_verb_prefixes.query('right_stem_cand.str.contains("\?")').iterrows():
    x = row.isv
    r, p = bite_all_prefixes_off(x, '')
    if p == 'do’':
        continue
    # print(x, p, r, morph.word_is_known(r))
    p2 = p.replace('’', "")
    for len_cand in range(1, len(p2)):
        word_cand = p2[-len_cand:] + r
        if morph.word_is_known(word_cand):
            partial_verb_prefixes.loc[i, "right_stem_cand"] = word_cand
            print(x, r, '->', word_cand)
            break



destabilizovati tabilizovati -> stabilizovati
doråzuměti měti -> uměti
dosaditi aditi -> saditi
dosaditi aditi -> saditi
dosaditi aditi -> saditi
dosęgati ęgati -> sęgati
dosęgnųti ęgnųti -> sęgnųti
dostavati tavati -> stavati
dostavati tavati -> stavati
dostavati tavati -> stavati
dostaviti taviti -> staviti
dostaviti taviti -> staviti
dostavjati tavjati -> stavjati
dostavjati tavjati -> stavjati
dostigati tigati -> stigati
dostigati tigati -> stigati
dostignųti tignųti -> stignųti
dostignųti tignųti -> stignųti
dověděti ěděti -> věděti
dověriti ěriti -> věriti
dověriti ěriti -> věriti
dověrjati ěrjati -> ověrjati
dověrjati ěrjati -> ověrjati
dověrjati ěrjati -> ověrjati
dovezti ezti -> vezti
dovoziti iti -> voziti
izdojiti jiti -> dojiti
iznajdti dti -> najdti
iznajęti ęti -> jęti
izobraziti iti -> raziti
izorati rati -> orati
izpovědati ědati -> povědati
izpověděti ěděti -> věděti
izprobovati bovati -> probovati
izprositi iti -> prositi
izraziti iti -> raziti
izraziti iti -> raziti


posěkati ěkati -> sěkati
posěkti ěkti -> sěkti
posęsti ęsti -> sęsti
poskųpiti kųpiti -> skųpiti
poslati lati -> slati
poslědovati lědovati -> slědovati
poslizgnųti lizgnųti -> slizgnųti
poslušati lušati -> slušati
posmotriti motriti -> smotriti
posȯvětovati ětovati -> sȯvětovati
pospati pati -> spati
pospěšiti pěšiti -> spěšiti
postanoviti tanoviti -> stanoviti
postarati tarati -> starati
postarěti tarěti -> starěti
postaviti taviti -> staviti
postaviti taviti -> staviti
postavjati tavjati -> stavjati
postavjati tavjati -> stavjati
postigati tigati -> stigati
postignųti tignųti -> stignųti
postlati tlati -> stlati
postųpati tųpati -> stųpati
postųpiti tųpiti -> stųpiti
posvęćati ęćati -> osvęćati
posvědčiti ědčiti -> svědčiti
posvědčiti ědčiti -> svědčiti
posvętiti ętiti -> svętiti
potręsati tręsati -> otręsati
poučiti čiti -> učiti
považiti ažiti -> važiti
pověděti ěděti -> věděti
pověriti ěriti -> věriti
pověriti ěriti -> věriti
pověrjati ěrjati -> ověrjati
pověrjati ěrjati -> ověrj

In [77]:
for x in ['okolorovati', 'izbombardovati', 'procitovati', 'prěprogramovati', 'zaprojektovati']:
    r, p = bite_all_prefixes_off(x, '')
    print(x, p, r, morph.word_is_known(r))
    p2 = p.replace('’', "")
    if not morph.word_is_known(r):
        for len_cand in range(len(p2) - 1):
            word_cand = p2[-len_cand:] + r
            print(word_cand, morph.word_is_known(word_cand))



okolorovati o’ kolorovati True
izbombardovati iz’ bombardovati True
procitovati pro’ citovati True
prěprogramovati prě’pro’ gramovati False
prěprogramovati True
ogramovati False
rogramovati False
programovati True
ěprogramovati False
zaprojektovati za’pro’ jektovati False
zaprojektovati True
ojektovati False
rojektovati False
projektovati True


In [175]:
BEG = partial_verb_prefixes.left_stem_cand.unique()


In [288]:
def add_derived_nouns(ending, replacement=''):
    tmp_df = slovnik[slovnik.partOfSpeech.apply(infer_pos) == 'noun'].copy()
    tmp_df['repl'] = tmp_df.isv.str.replace(ending + "$", replacement)
    tmp_df = tmp_df.query(
        "isv not in @BEG and repl in @BEG"
    ).copy()
    for i, row in tmp_df.iterrows():
        repl = row.repl
        matches = partial_verb_prefixes.query("left_stem_cand == @repl")

        partial_verb_prefixes.loc[matches.index, "derived_nouns"] = partial_verb_prefixes.loc[matches.index, "derived_nouns"] + "|" + row.isv

    return tmp_df.isv.values.tolist()

In [257]:
matches = partial_verb_prefixes.query("left_stem_cand == @repl")

partial_verb_prefixes.loc[matches.index, "derived_nouns"] = partial_verb_prefixes.loc[matches.index, "derived_nouns"] + "|" + row.isv

In [259]:
partial_verb_prefixes.sort_values(by='derived_nouns')

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
5,11,abdikovati,NaN,v.intr. ipf.,1,abdicate,NaN,I,отрекаться,адракацца,...,{отрекаться},{абдикирам},?,?,abdikovati,abdikovati,abdik,abdikovati,,
12541,30170,råzgladiti,NaN,v.tr. pf.,NaN,smooth out,NaN,NaN,разгладить,"разгладзіць, распрасаваць",...,{разгладить},{!изглаждане на},råz’,gladiti,gladiti,råzgladiti,råzglad,gladiti,,
12540,30169,råzgarjati,NaN,v.refl. ipf.,1,"flare up, be ablaze",NaN,NaN,разгораться,"разгарацца, распальвацца",...,{разгораться},"{!ще пламне, ще изгори}",råz’,garjati sę,,råzgarjati sę,råzgar,?garjati?,,
12539,30168,råzganjati,NaN,v.tr. ipf.,NaN,"disperse, dispel",NaN,NaN,разгонять,разганяць,...,{разгонять},"{!да се разпръсне, да разсее}",råz’,ganjati,,råzganjati,råzgan,?ganjati?,,
12538,33765,råzgadyvati,NaN,v.tr. ipf.,NaN,"unriddle, solve, fathom out",NaN,NaN,"разгадывать, отгадывать","разгадваць, адгадваць",...,"{разгадывать, отгадывать}","{!разгадаят, да реши да се разбере}",råz’,gadyvati,gadati,råzgadyvati,råzgad,?gadyvati?,gad,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,22666,obråtiti,NaN,v.tr. pf.,2,"rotate, revolve (tr.)",uk pl yu,NaN,"повернуть, обернуть, оборотить","павярнуць, пакруціць, абвінуць, абгарнуць, абк...",...,"{оборотить, обернуть, повернуть}","{!върти, върти (tr.)}",?,?,,obråtiti,obråt,?råtiti?,obråt,
8514,3209,obråtiti,NaN,v.tr. pf.,1,turn around,NaN,NaN,"повернуть, обернуть, оборотить","павярнуць, пакруціць, абвінуць, абгарнуць, абк...",...,"{оборотить, обернуть, повернуть}",{!обърни се},?,?,,obråtiti,obråt,?råtiti?,obråt,
8536,16844,obrěmeniti,NaN,v.tr. pf.,1,burden,ru pl sk j,NaN,"обременить, отяготить","абцяжарваць, абцяжаць",...,"{обременить, отяготить}",{!тежест},?,?,obrěmeniti,obrěmeniti,obrěmen,?rěmeniti?,,
17921,-37004,zapečętyvati,NaN,v.tr.ipf.,NaN,seal,NaN,NaN,"запечатывать, опечатывать","запячатваць, апячатваць",...,"{запечатывать, опечатывать}",{!запечатвам},?,?,,zapečętyvati,zapečęt,?pečętyvati?,,


In [ ]:
result = []

result += add_derived_nouns("ťje")
result += add_derived_nouns("je")
result += add_derived_nouns("ńje")
result += add_derived_nouns("ťje")
result += add_derived_nouns("ŕje")


for letter, letter_repl in ['ďd', 'ľl', 'ńn', 'ŕr', 'śs', 'ťt', 'źz']:
    result += add_derived_nouns(letter + "je", letter_repl)
    print(letter, len(result))

result

In [ ]:
result = []
for letter, letter_repl in ['čk', 'šh']:
    result += add_derived_nouns(letter + "je", letter_repl)
    print(letter, len(result))

result += add_derived_nouns('nik')
print(len(result))
result += add_derived_nouns('nica')
print(len(result))

result += add_derived_nouns('ec')
print(len(result))
result += add_derived_nouns('ica')
print(len(result))

result += add_derived_nouns('stvo')
print(len(result))
result += add_derived_nouns('išče')
print(len(result))
result += add_derived_nouns('osť')
print(len(result))
result += add_derived_nouns('ač')
print(len(result))
result += add_derived_nouns('aŕ')
print(len(result))


result

## конец архива

In [99]:

possible_prefixes = {
    "ne", "bez", "naj",
    "do", "iz", "na", "nad", "ne", "o", "ob", "obez", "od", "po", "pod", 
    "prě", "prěd", "pri", "pro", 
    "raz", "råz",
    "de", 
    "s", "sȯ",
    "so", "su", "vo", "voz", 
    "sų", "u", "v", 
    "vȯz", "vȯ",
    "vy", "za"
}
possible_prefixes = sorted(list(possible_prefixes), key=len, reverse=True)

possible_suffixes = [" sę", "iti", "ěti", 'nųti', 'ov', 'yvati', 'ati', 'j', 'ivati']

possible_suffixes = sorted(list(possible_suffixes), key=len, reverse=True)



def bite_all_prefixes_off(word, verb_nest):

    can_continue = True
    prefixes = []

    while can_continue:
        can_continue = False
        for pref in possible_prefixes:
            if word.startswith(pref):
                # print(pref, word, word[len(pref):])
                new_word = word[len(pref):]
                if new_word.endswith(verb_nest):
                    word = new_word
                    prefixes.append(pref)
                    can_continue = True
                    break
    return word, "’".join(prefixes) + "’"


def bite_all_suffixes_off(word, verb_nest):

    can_continue = True
    suffixes = []

    while can_continue:
        can_continue = False
        for pref in possible_suffixes:
            if word.endswith(pref):
                new_word = word[:-len(pref)]
                if new_word.startswith(verb_nest):
                    word = new_word
                    suffixes.append(pref)
                    can_continue = True
                    break
    return word, "+".join(reversed(suffixes)) + "+"



In [100]:
tmp_df.left_stem_cand.apply(lambda x: 
   bite_all_prefixes_off(x, '')[0] if morph.word_is_known(bite_all_prefixes_off(x, '')[0]) else "?" + bite_all_prefixes_off(x, '')[0] + "?"
)

4159         pepel
7497        ?tråž?
8267         ?agr?
8368         ?lač?
8438           rod
8481           ?ž?
8675           čar
8684           ček
8978         ?rač?
9000         ?roč?
9185        ?graž?
9450      ?pustaš?
9559        ?irot?
9775           ?d?
10418        ?nur?
10915       ?rědk?
11371          ?š?
12136          tir
12200     ?tivrěč?
12763          pěn
12828          ?n?
12929      ?tȯlst?
12942        ?tro?
13382      ?amoub?
13453    ?ebesmŕt?
13621         hvat
13711       ?karž?
13793          kub
13794      ?kubti?
13808       ?kvrč?
13966          ?m?
14006        ?myk?
14014        ?něd?
14103         ?mk?
14263       ?pręg?
14366        ?tač?
14386           ta
14523        ?trč?
14700          ?t?
14738        ?ědč?
14839         ?ŕb?
15636        ?gaš?
15683          kųs
15782          per
15849           il
15988     ?ěkověč?
16089     ?aljcev?
16132           až
16192          ?ě?
16205         ?el?
16258        ?erb?
16424         ?is?
16600       

In [101]:
tmp_df.left_stem_cand.apply(lambda x: 
   bite_all_prefixes_off(x, '')[0] + "a" if morph.word_is_known(bite_all_prefixes_off(x, '')[0] + "a") else "?" + bite_all_prefixes_off(x, '')[0] + "?"
)

4159        pepela
7497        ?tråž?
8267         ?agr?
8368         ?lač?
8438          roda
8481           ?ž?
8675          čara
8684          čeka
8978         ?rač?
9000         ?roč?
9185        ?graž?
9450      ?pustaš?
9559        ?irot?
9775            da
10418        ?nur?
10915        rědka
11371          ?š?
12136         tira
12200     ?tivrěč?
12763         pěna
12828           na
12929       tȯlsta
12942        ?tro?
13382      ?amoub?
13453    ?ebesmŕt?
13621        hvata
13711       ?karž?
13793         kuba
13794      ?kubti?
13808       ?kvrč?
13966          ?m?
14006        ?myk?
14014        ?něd?
14103         ?mk?
14263       ?pręg?
14366        ?tač?
14386         ?ta?
14523        ?trč?
14700           ta
14738        ?ědč?
14839         ?ŕb?
15636        ?gaš?
15683         kųsa
15782         pera
15849          ila
15988     ?ěkověč?
16089     ?aljcev?
16132         ?až?
16192          ?ě?
16205         ?el?
16258        ?erb?
16424         ?is?
16600       

In [35]:
# is_known = partial_verb_prefixes.query("base_verb == '' and not isv.str.contains(' ') and left_stem_cand != isv").left_stem_cand.apply(
# is_known = partial_verb_prefixes.query("not isv.str.contains(' ') and left_stem_cand != isv and (base_verb == '' or base_verb == 'isv')").left_stem_cand.apply(

is_known = partial_verb_prefixes.query("not isv.str.contains(' ') and left_stem_cand != isv and (base_verb == '' or base_verb == isv)").left_stem_cand.apply(
    lambda stem: any(
        (morph.word_is_known(x) and any(v.tag.POS == "NOUN" for v in morph.parse(x)))
        for x in [stem, stem + "a"]
    )
)

tmp = is_known[is_known]

partial_verb_prefixes.loc[tmp.index]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
59,4540,adresovati,NaN,v.tr. ipf.,address,I,verb,?,?,adresovati,adresovati,adres,adresovati,,
120,24148,akcentovati,NaN,v.tr. ipf.,"accentuate, stress (pronunciation)",I,verb,?,?,akcentovati,akcentovati,akcent,akcentovati,,
285,3969,analizovati,NaN,v.tr. ipf.,"analyse, analyze",I,verb,?,?,analizovati,analizovati,analiz,analizovati,,
369,122,areštovati,NaN,v.tr. ipf.,arrest,I,verb,?,?,areštovati,areštovati,arešt,areštovati,,
375,19640,argumentovati,NaN,v.intr. ipf.,argue,I,verb,?,?,argumentovati,argumentovati,argument,argumentovati,|argumentacija,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17792,19402,žariti,NaN,v.tr. ipf.,"incandesce, heat to a glow",NaN,verb,?,?,žariti,žariti,žar,žariti,,
17797,19406,žartovati,NaN,v.intr. ipf.,joke,D,verb,?,?,žartovati,žartovati,žart,žartovati,,
17811,1047,želati,NaN,v.tr. ipf.,"wish, desire",NaN,verb,?,?,želati,želati,žel,želati,,
17827,3399,ženiti,NaN,v.refl. ipf.,"marry (with a woman), get married (with a woman)",NaN,verb,?,?,ženiti,ženiti sę,žen,ženiti,,


In [48]:
for i, row in partial_verb_prefixes.loc[tmp.index].iterrows():
    stem = row.left_stem_cand
    base_noun_cands = []
    for x in [stem, stem + "a"]:
        variants = morph.parse(x)
        variants = [v for v in variants if v.tag.POS == "NOUN"]
        variants = [v for v in variants if v.normal_form == v.word and morph.word_is_known(v.normal_form)]
        if variants:
            variants = set([("!" if v.word != x else "") + v.word for v in variants])
            base_noun_cands += list(variants)
    if base_noun_cands:
        print(row.isv, row.base_verb, stem, base_noun_cands)
        partial_verb_prefixes.loc[i, "base_noun"] = "|".join(base_noun_cands)


adresovati adresovati adres ['adres', 'adresa']
akcentovati akcentovati akcent ['akcent']
analizovati analizovati analiz ['analiza']
areštovati areštovati arešt ['arešt']
argumentovati argumentovati argument ['argument']
arhivovati arhivovati arhiv ['arhiv']
atakovati atakovati atak ['ataka']
avansovati avansovati avans ['avans']
avtostopovati avtostopovati avtostop ['avtostop']
balansovati balansovati balans ['balans']
balzamovati balzamovati balzam ['balzam']
barikadovati barikadovati barikad ['barikada']
barviti barviti barv ['barva']
bazovati bazovati baz ['baza']
běgati běgati běg ['běg']
besědovati besědovati besěd ['besěda']
běsiti běsiti běs ['běs']
blazniti blazniti blazn ['blazn']
blaznovati blaznovati blazn ['blazn']
blěskati blěskati blěsk ['blěsk']
blokovati blokovati blok ['blok']
blųditi blųditi blųd ['blųd']
bojati bojati bo ['boa']
bojkotovati bojkotovati bojkot ['bojkot']
boriti boriti bor ['bor']
bråniti bråniti brån ['bråna']
bratati bratati brat ['brat']
bråzditi b

In [50]:
partial_verb_prefixes[partial_verb_prefixes.left_stem_cand == "zamk"]
partial_verb_prefixes.loc[17302, "base_noun"] = "zamȯk"


In [ ]:
ć č ď ė ę ě ľ ń ŕ ś š ť ų ź ž ȯ ʒ

In [ ]:
čk šh 

In [17]:
partial_verb_prefixes.query("isv.str.contains('stvo')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
4592,14605,jestvovati,NaN,v.intr. ipf.,exist,NaN,verb,?,?,jestvovati,jestvovati,jestv,jestvovati,,
9061,36666,odsųtstvovati,NaN,v.intr. ipf.,"be absent, be away",NaN,verb,?,?,sųtstvovati,odsųtstvovati,odsųtstv,sųtstvovati,,
11859,23860,prisųtstvovati pri,(+6),v.intr. ipf.,"attend, be present at",NaN,verb,?,?,,prisųtstvovati pri,prisųtstvovati pri,?tstvovati pri?,,
14056,34940,sȯčuvstvovati,NaN,v.intr. ipf.,"sympathize, commiserate, feel compassion",NaN,verb,sȯ’,čuvstvovati,čuvati,sȯčuvstvovati,sȯčuvstv,?čuvstvovati?,,
14593,6307,stvoriti,NaN,v.tr. pf.,"create, make, form",NaN,verb,?,?,tvoriti,stvoriti,stvor,tvoriti,,
14707,36821,sųtstvovati,NaN,v.intr. ipf.,exist,NaN,verb,?,?,sųtstvovati,sųtstvovati,sųtstv,?tstvovati?,,
15578,2948,učęstvovati,NaN,v.intr. ipf.,participate,NaN,verb,?,?,,učęstvovati,učęstv,?čęstvovati?,,
17048,9090,vzajemodějstvovati,NaN,v.intr. ipf.,interact,NaN,verb,?,?,dějati,vzajemodějstvovati,vzajemodějstv,?jemodějstvovati?,,


In [170]:
tmp_df = partial_verb_prefixes.query("base_verb == '' and base_noun == '' and not isv.str.contains(' ') and not isv.str.contains('/')").copy()
# tmp_df['en_cognate'] = tmp_df.en.apply(KNOWN_EN_DICT.get)

In [171]:
tmp_df.shape, partial_verb_prefixes.shape

((4, 15), (4985, 15))

In [172]:
tmp_df#.head(37)

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
12828,18617,råzraznjati,NaN,v.tr. ipf.,distinguish,NaN,verb,råz’,raznjati,,råzraznjati,råzrazn,?njati?,,
12942,17026,råztrojiti,NaN,v.tr. pf.,split in three,NaN,verb,råz’,trojiti,,råztrojiti,råztro,?trojiti?,,
14366,7236,stačiti,NaN,v.intr. ipf.,suffice,NaN,verb,?,?,,stačiti,stač,?tačiti?,,
14523,938,strčiti,NaN,v.intr. ipf.,"protrude, stick out",NaN,verb,?,?,,strčiti,strč,?trčiti?,,


In [ ]:
råzny

In [174]:
partial_verb_prefixes.query("isv.str.contains('råzdv')")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
12534,18592,råzdvajati,NaN,v.tr. ipf.,divide in two,NaN,verb,råz’,dvajati,dvojiti,råzdvajati,råzdva,?dvajati?,,
12536,18593,råzdvojiti,NaN,v.tr. pf.,divide in two,NaN,verb,råz’,dvojiti,dvojiti,råzdvojiti,råzdvo,dvojiti,,


In [176]:
MANUAL_FIXES = {
"råzraznjati": "råzniti",
"strčiti": ERR_SYMB + "trčati",
"råztrojiti": "trojiti",
"stačiti": "stačiti",
}

In [167]:
IDX = partial_verb_prefixes.query("isv.str.contains('pręg')").index
partial_verb_prefixes.loc[IDX, "base_verb"] = "pręgti"



In [168]:
MANUAL_FIXES = {
    "svědčiti": "věděti",
    "skubnųti": "skubti",
    "skubti": "skubti",
    "skvrčati": "skvrčati",
    "skaržiti": "skarga",
    "posrědkovati": "srěd",
"vrěščati": "vrěsk",
"vrčati": "vrčati",
"vojevati": "vojna",
"verbovati": "verbovati",
"vějati": "vějati",
"važiti": "vagati",
"usilovati": "sila",
"uperiti": "pero",
"směti": "směti",
"podnurjati": ERR_SYMB + "nuriti",
"protivrěčiti": "rěč",
"stanųti": "stan",
"obožati": "bog",
"oblačati": "oblåčiti",
"očekyvati": "čekati",
"svŕběti": "svŕběti",
"snědati": "snědati",
    
}


In [126]:
tmp_df.isv.values

array(['oblačati', 'obožati', 'očekyvati', 'ogražati', 'opustašati',
       'osirotiti', 'ovdověti', 'podnurjati', 'posrědkovati',
       'prědvyšati', 'protivrěčiti', 'råzpěniti', 'råzraznjati',
       'råztȯlstěti', 'råztrojiti', 'samoubiti', 'sebesmŕtiti',
       'shvatiti', 'skaržiti', 'skubnųti', 'skubti', 'skvrčati', 'směti',
       'smykati', 'snědati', 'sȯmknųti', 'spręgati', 'stačiti', 'stanųti',
       'strčiti', 'sųt', 'svědčiti', 'svŕběti', 'ugašati', 'ukųsiti',
       'uperiti', 'usilovati', 'uvěkověčiti', 'valjcevati', 'važiti',
       'vějati', 'velěti', 'verbovati', 'visěti', 'vojevati',
       'vȯzkypyvati', 'vȯzpitati', 'vrčati', 'vrěščati', 'zaklinati',
       'zakliniti', 'zaklinovati'], dtype=object)

In [ ]:
opustěti opustošiti pustošiti # opustiti

In [151]:
partial_verb_prefixes.query('isv.str.contains("pust[ao]š") or isv.str.contains("pust[ěi]")').base_verb.unique()

array(['pustiti', '', 'pustěti', 'pustošiti'], dtype=object)

In [159]:
IDX = partial_verb_prefixes.query('isv.str.contains("vys") or isv.str.contains("vyš")').head(9).index

partial_verb_prefixes.loc[IDX, "base_verb"] = ERR_SYMB + "vysiti"

In [160]:
partial_verb_prefixes
partial_verb_prefixes.query('isv.str.contains("kųsa")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
5712,17718,kųsati,NaN,v.tr. ipf.,bite,NaN,verb,?,?,kųsati,kųsati,kųs,kųsati,,kųs


In [155]:
#slovnik.query('isv.str.contains("pust")')

In [161]:
MANUAL_FIXES = {
    "ogražati": "groziti",
    "opustašati": "pustošiti",
    "vȯzkypyvati": "kypěti",
    "valjcevati": "val",
    "ukųsiti": "kųsati"
}


In [143]:
partial_verb_prefixes.query('isv.str.contains("pitati")')

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
16743,19149,vȯzpitati,NaN,v.tr. pf.,"bring up, nurture, raise, rear",NaN,verb,vȯz’,pitati,,vȯzpitati,vȯzpit,?pitati?,,


In [138]:
MANUAL_FIXES = {
    "velěti": "liti",
    "råztȯlstěti": "tȯlsty",
    "uvěkověčiti": "věk",
    "ugašati": "gasnųti",
    "osirotiti": "sirota",
    "ovdověti": "vdova",
    "råzpěniti": "pěna",
    "samoubiti": "biti",
    "sebesmŕtiti": "smŕť",
    "shvatiti": "hvat",
    "smykati": "sȯmknųti",
    "sȯmknųti": "sȯmknųti",
    "sųt": "byti",
    "visěti": "visěti",
    "vȯzpitati": ERR_SYMB + "pitati",
    "sųt": "byti",
    "zakliniti": "klin",
    "zaklinovati": "klin",
    "zaklinati": "klęti"
}    

In [177]:
for k, v in MANUAL_FIXES.items():
    IDX = partial_verb_prefixes.query("isv == @k").index
    if v == ERR_SYMB + "pitati":
        colname = "base_verb"
    else:
        pos = morph.parse(v.replace(ERR_SYMB, ""))[0].tag.POS
        if pos in {"ADJF", "NOUN"}:
            colname = "base_noun"
        if pos in {"VERB"}:
            colname = "base_verb"
    partial_verb_prefixes.loc[IDX, colname] = v
    print(k, v, colname)
    

råzraznjati råzniti base_verb
strčiti †trčati base_verb
råztrojiti trojiti base_verb
stačiti stačiti base_verb


In [197]:
list(KV.values)

array(['abdikovati', 'abdikovati'], dtype=object)

In [200]:
KV = partial_verb_prefixes.query("base_verb != '' or base_noun != ''")[['isv', 'base_verb']]
KV = dict(list(KV.values))

tmp = partial_verb_prefixes.query("base_verb == '' and base_noun == ''").isv.str.partition(" ")[0].apply(lambda x: x in KV)

IDX = partial_verb_prefixes.query("base_verb == '' and base_noun == ''")[tmp].index
partial_verb_prefixes.loc[IDX, "base_verb"] = partial_verb_prefixes.loc[IDX, "isv"].str.partition(" ")[0].apply(KV.get)

In [201]:
partial_verb_prefixes.query("base_verb == '' and base_noun == ''")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
1185,22979,bųde/bųdųt,NaN,v.aux. ipf.,there will be,NaN,verb,?,?,,bųde/bųdųt,bųde/bųdųt,?bųde/bųdųt?,,
1985,25269,doběgati do,(+2),v.intr. ipf.,reach (by running),NaN,verb,?,?,,doběgati do,doběgati do,?běgati do?,,
2269,35918,dovesti do/k,(dovede),v.tr. pf.,lead to,NaN,verb,?,?,,dovesti do/k,dovesti do/k,?esti do/k?,,
2275,35917,dovoditi do/k,NaN,v.tr. ipf.,lead to,NaN,verb,?,?,,dovoditi do/k,dovoditi do/k,?diti do/k?,,
3685,3325,iměti dělo s,NaN,v.ipf.,deal with,NaN,verb,?,?,,iměti dělo s,iměti dělo s,?iměti dělo s?,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16679,5062,vȯzdavati česť,NaN,v.ipf.,"pay honour, pay honor",NaN,verb,vȯz’,davati česť,,vȯzdavati česť,vȯzdavati česť,?davati česť?,,
17367,32910,zaphati v něčto,NaN,v.tr. pf.,stuff into,NaN,verb,?,?,,zaphati v něčto,zaphati v něčto,?phati v něčto?,,
17368,32909,zapihati v něčto,NaN,v.tr. ipf.,stuff into,NaN,verb,za’,phati/pihati,,zapihati v něčto,zapihati v něčto,?pihati v něčto?,,
17553,20321,zavisěti od,(zavisi),v.intr. ipf.,depend on,NaN,verb,?,?,,zavisěti od,zavisěti od,?isěti od?,,


In [ ]:


obvodniti -> dniti

osušiti -> šiti
udariti -> dariti
zadovoliti -> liti

žiti
byti
byvati


In [238]:
IDX1 = partial_verb_prefixes.query("base_verb.str.contains('^liti') and isv.str.contains('sol')").index
IDX2 = partial_verb_prefixes.query("base_verb.str.contains('^liti') and (isv.str.contains('vel') or isv.str.contains('vol'))").index


In [241]:
partial_verb_prefixes.loc[IDX1, "base_verb"] = "soliti"

In [242]:
partial_verb_prefixes.loc[IDX2, "base_verb"] = "voliti"

In [246]:


partial_verb_prefixes.loc[
    partial_verb_prefixes.query("base_verb.str.contains('^dniti')").index, 
    "base_verb"
] = "vodniti"

partial_verb_prefixes.loc[[12516, 1973], "base_verb"] = "dniti"

partial_verb_prefixes.loc[
    partial_verb_prefixes.query("base_verb.str.contains('^vodniti')").index, 
    "base_noun"
] = "voda"


In [259]:
partial_verb_prefixes.query("base_verb.str.contains('pisati')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
7331,958,napisati,(napiše),v.tr. pf.,"write, write down",NaN,verb,?,?,pisati,napisati,napis,pisati,,
8919,29136,odpisati,(odpiše),v.tr. pf.,write off,NaN,verb,?,?,pisati,odpisati,odpis,pisati,,
8920,29137,odpisati,(odpiše),v.tr. pf.,"reply (in writing), write back",NaN,verb,?,?,pisati,odpisati,odpis,pisati,,
8921,34295,odpisyvati,NaN,v.tr. ipf.,write off,NaN,verb,?,?,pisati,odpisyvati,odpis,?pisyvati?,,
8922,34296,odpisyvati,NaN,v.tr. ipf.,"reply (in writing), write back",NaN,verb,?,?,pisati,odpisyvati,odpis,?pisyvati?,,
9379,2,opisati,(opiše),v.tr. pf.,describe,NaN,verb,?,?,pisati,opisati,opis,pisati,,
9380,686,opisyvati,NaN,v.tr. ipf.,describe,NaN,verb,?,?,pisati,opisyvati,opis,?pisyvati?,,
10169,15680,pisati nanovo,(piše),v.ipf.,rewrite,NaN,verb,?,?,pisati,pisati nanovo,pisati nanovo,?pisati nanovo?,,
10170,141,pisati,(piše),v.tr. ipf.,write,NaN,verb,?,?,pisati,pisati,pis,pisati,,
10439,3086,podpisati,(podpiše),v.tr. pf.,"sign, subscribe",NaN,verb,?,?,pisati,podpisati,podpis,pisati,|podpiska|podpisnik,


In [260]:
partial_verb_prefixes.query("base_verb.str.contains('pisati')").en.unique()


array(['write, write down', 'write off', 'reply (in writing), write back',
       'describe', 'rewrite', 'write', 'sign, subscribe', 'prescribe',
       'rewrite, copy', 'attribute, ascribe', 'list', 'enter (data)',
       'register', 'record, save (on a computer), write down', 'note'],
      dtype=object)

In [265]:
for k, g in partial_verb_prefixes.groupby("en"):
    if k[0] != "#" and len(g) > 1 and len(g.base_verb.unique()) > 1:
        print(k, g.base_verb.unique())
        break
g

abandon ['' 'odstųpati' 'pustiti' 'pušćati']


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
8793,15415,odidti od,(odide; odšel),v.tr. pf.,abandon,NaN,verb,?,?,,odidti od,odidti od,?idti od?,,
9047,15417,odstųpiti od,NaN,v.tr. pf.,abandon,NaN,verb,?,?,odstųpati,odstųpiti od,odstųpiti od,?tųpiti od?,,
9452,10246,opustiti,NaN,v.tr. pf.,abandon,NaN,verb,?,?,pustiti,opustiti,opust,pustiti,,
9457,10250,opušćati,NaN,v.tr. ipf.,abandon,NaN,verb,?,?,pušćati,opušćati,opušć,pušćati,,


In [266]:
len(partial_verb_prefixes.base_verb.unique())

1386

In [273]:
vc = partial_verb_prefixes.base_verb.value_counts()

vc[vc == 1]

žariti             1
zějati             1
postulovati        1
zvųčati            1
zloradovati        1
                  ..
konkurovati        1
konservovati       1
konstruovati       1
kontrabandovati    1
žužati             1
Name: base_verb, Length: 446, dtype: int64

In [274]:
vc[vc > 1]

              207
praviti        34
biti           31
kazati         31
ględěti        30
             ... 
kymati          2
zavojevati      2
provŕgati       2
kypěti          2
pųkati          2
Name: base_verb, Length: 940, dtype: int64

In [269]:
partial_verb_prefixes[partial_verb_prefixes.base_verb.str.contains('ć')].base_verb.unique()

array(['pušćati', 'vraćati', 'moćněti', 'noćevati', 'oběćati', 'obobćati',
       'obraćati', 'očišćati', 'prašćati', 'osvěćati', 'osvęćati',
       'strěćati', 'upȯlnomoćiti', 'utekųćiniti'], dtype=object)

In [267]:
len(partial_verb_prefixes.base_verb.str.replace("šćati", "stiti").str.replace('ćati', 'titi').unique())

1384

In [258]:
partial_verb_prefixes.derived_nouns.value_counts()

                        4370
|zadŕžnik                 12
|odstųpnik|odstųpnik      10
|dostavka                  7
|zapiska|zapisnik          6
                        ... 
|koncentracija             1
|komunikacija              1
|komplikacija              1
|kompensacija              1
|žrec|žrica                1
Name: derived_nouns, Length: 404, dtype: int64

In [249]:
partial_verb_prefixes.loc[[12516, 1973], "base_noun"] = "denj"


In [247]:
partial_verb_prefixes.query("base_verb.str.contains('dniti')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
1973,17259,dniti,NaN,v.refl. ipf.,dawn,NaN,verb,?,?,dniti,dniti sę,dn,dniti,,
4194,26231,izpråzdniti,NaN,v.tr. pf.,"empty, clear out, vacate",NaN,verb,?,?,izpråzdniti,izpråzdniti,izpråzdn,?pråzdniti?,,
4195,31723,izpråzdnjati,NaN,v.tr. ipf.,"empty, clear out, vacate",NaN,verb,?,?,izpråzdniti,izpråzdnjati,izpråzdn,?pråzdnjati?,,
7582,22551,navodniti,NaN,v.tr. pf.,irrigate,NaN,verb,?,?,vodniti,navodniti,navodn,dniti,|navodnica,voda
7584,22550,navodnjati,NaN,v.tr. ipf.,irrigate,NaN,verb,?,?,vodniti,navodnjati,navodn,?dnjati?,|navodnica,voda
8629,4488,obvadnjati,NaN,v.tr. ipf.,"irrigate, water",NaN,verb,?,?,vodniti,obvadnjati,obvadn,?adnjati?,,voda
8649,4489,obvodniti,NaN,v.tr. pf.,"irrigate, water",NaN,verb,?,?,vodniti,obvodniti,obvodn,dniti,,voda
9091,10074,odvadnjati,NaN,v.tr. ipf.,dehydrate,NaN,verb,?,?,vodniti,odvadnjati,odvadn,?adnjati?,,voda
9111,10093,odvodniti,NaN,v.tr. pf.,dehydrate,NaN,verb,?,?,vodniti,odvodniti,odvodn,dniti,,voda
9403,29532,opozdniti,NaN,v.refl. pf.,be late,NaN,verb,?,?,opozdniti,opozdniti sę,opozdn,?zdniti?,,


In [227]:
partial_verb_prefixes.query("base_verb.str.contains('věrit')")


partial_verb_prefixes.loc[
    partial_verb_prefixes.query("base_verb.str.contains('věrit')").index,
    "base_verb"
] = "věriti"


In [256]:
    partial_verb_prefixes.query("base_verb.str.contains('^ti$')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
2154,3813,donesti,NaN,v.tr. pf.,"report, inform",NaN,verb,?,?,ti,donesti,donesti,ti,,
4130,5459,iznesti,NaN,v.tr. pf.,carry out,NaN,verb,?,?,ti,iznesti,iznesti,ti,,
7297,28077,nanesti,NaN,v.tr. pf.,bring,NaN,verb,?,?,ti,nanesti,nanesti,ti,,
7367,28129,naprěti,(napre),v.intr. pf.,"press, push against",NaN,verb,?,?,ti,naprěti,napr,ti,,
7965,20547,nesti jajca,NaN,v.tr. ipf.,lay eggs,NaN,verb,?,?,ti,nesti jajca,nesti jajca,?ti jajca?,,
7966,1161,nesti,NaN,v.tr. ipf.,"carry (on foot), bear",NaN,verb,?,?,ti,nesti,nesti,ti,,
8445,28846,obnesti,NaN,v.tr. pf.,carry round,NaN,verb,?,?,ti,obnesti,obnesti,ti,,
8446,34078,obnesti,NaN,v.tr. pf.,wear out,NaN,verb,?,?,ti,obnesti,obnesti,ti,,
8622,28970,obuti,(obuje),v.refl. pf.,put on one’s shoes,NaN,verb,?,?,ti,obuti sę,obuti,ti,,
8624,28972,obuvati,NaN,v.refl. ipf.,put on one’s shoes,NaN,verb,?,?,ti,obuvati sę,obuv,?ati?,|obuvka|obuvnik,


In [221]:

partial_verb_prefixes.loc[
    partial_verb_prefixes.query("isv.str.contains('gl[ęě]d')").index,
    "base_verb"
] = "ględěti"



In [215]:
TMP_SET = partial_verb_prefixes.loc[
    partial_verb_prefixes.query("isv.str.contains('niziti')").index,
    "base_verb"
].values

In [216]:
partial_verb_prefixes.query("base_verb in @TMP_SET")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
8447,19932,obniziti,NaN,v.tr. pf.,lower,NaN,verb,?,?,obniziti,obniziti,obniz,?niziti?,,
8448,19930,obnižati,NaN,v.tr. ipf.,lower,NaN,verb,?,?,obniziti,obnižati,obniž,?nižati?,,
10753,34860,poniziti,NaN,v.tr. pf.,"humiliate, humble",NaN,verb,?,?,poniziti,poniziti,poniz,?niziti?,,
10756,34858,ponižati,NaN,v.tr. ipf.,"humiliate, humble",NaN,verb,?,?,poniziti,ponižati,poniž,?nižati?,,
14028,5636,sniziti,NaN,v.tr. pf.,"reduce, lower",NaN,verb,?,?,sniziti,sniziti,sniz,?niziti?,,
14030,5635,snižati,NaN,v.tr. ipf.,"reduce, lower",NaN,verb,?,?,sniziti,snižati,sniž,?nižati?,|snižka,
15770,4508,uniziti,NaN,v.tr. pf.,"humiliate, humble",NaN,verb,?,?,uniziti,uniziti,uniz,?niziti?,,
15772,4507,unižati,NaN,v.tr. ipf.,"humiliate, humble",NaN,verb,?,?,uniziti,unižati,uniž,?nižati?,,


In [217]:
partial_verb_prefixes.loc[
    partial_verb_prefixes.query("base_verb in @TMP_SET").index,
    "base_verb"
] = ERR_SYMB + "niziti"

In [250]:
partial_verb_prefixes.to_pickle("verb_prefixes.pkl")

In [251]:
partial_verb_prefixes[['id', 'isv_orig', 'addition', 'partOfSpeech', 'en', 'genesis', 'base_verb', 'base_noun', 'derived_nouns', 
       'left_stem_cand',
       'right_stem_cand', ]].to_csv("verb_prefixes.csv")

In [270]:
partial_verb_prefixes.base_verb.value_counts().head(44)

            207
praviti      34
biti         31
kazati       31
ględěti      30
ti           30
dŕžati       29
pisati       27
javiti       26
davati       26
hoditi       26
věriti       23
voditi       23
staviti      23
idti         23
stavjati     22
jęti         22
zvati        22
liti         22
dati         22
kryti        21
pušćati      21
raziti       21
žiti         21
jehati       20
govoriti     20
pustiti      20
byti         20
pasti        20
tekti        19
saditi       19
nositi       18
stavati      18
brati        18
piti         18
měnjati      18
rvati        18
vesti        18
tvoriti      18
råsti        17
značiti      17
plyvti       17
tati         17
měšati       16
Name: base_verb, dtype: int64

In [ ]:
partial_verb_prefixes.to_excel("verb_prefixes.xlsx")

In [78]:
IDX = tmp_df[['isv', 'right_stem_cand', 'left_stem_cand', 'base_verb', 'base_noun']].index
tmp_df[['isv', 'right_stem_cand', 'left_stem_cand', 'base_verb', 'base_noun']]


,isv,right_stem_cand,left_stem_cand,base_verb,base_noun
1742,degradovati,?gradovati?,degrad,,
4131,izniknųti,?niknųti?,iznik,,
4159,izpepeliti,?pepeliti?,izpepel,,
4297,izsysati,?ysati?,izsys,,
4351,izvihnųti,?ihnųti?,izvih,,
...,...,...,...,...,...
16852,vrěščati,?rěščati?,vrěšč,,
16974,vulkanizovati,?lkanizovati?,vulkaniz,,
17230,zaklinati,?klinati?,zaklin,,
17231,zakliniti,?kliniti?,zaklin,,


In [79]:
tmp_df.isv.values

array(['degradovati', 'izniknųti', 'izpepeliti', 'izsysati', 'izvihnųti',
       'nakalati', 'nastråžiti', 'obagriti', 'oblačati', 'obnarodovati',
       'obȯzrěti', 'obožati', 'očariti', 'očekyvati', 'odračati',
       'odročiti', 'ogražati', 'opustašati', 'organizovati', 'osirotiti',
       'ovdověti', 'podnurjati', 'pogynati', 'posrědkovati', 'prědvyšati',
       'profesionalizovati', 'prostirati', 'prostirati', 'protivrěčiti',
       'råzkalati', 'råzpěniti', 'råzraznjati', 'råztȯlstěti',
       'råztrojiti', 'samoubiti', 'sebesmŕtiti', 'shvatiti',
       'signalizovati', 'simbolizovati', 'skaržiti', 'skubnųti', 'skubti',
       'skvrčati', 'směti', 'smykati', 'snědati', 'sȯmknųti', 'spręgati',
       'stačiti', 'standardizovati', 'stanųti', 'strčiti', 'sųt',
       'svědčiti', 'svŕběti', 'ugašati', 'ukųsiti', 'uperiti',
       'usilovati', 'uvěkověčiti', 'valjcevati', 'važiti', 'vějati',
       'velěti', 'verbovati', 'visěti', 'vojevati', 'vȯzkypyvati',
       'vȯzpitati', 'vrčati

In [80]:
'odračati', 'odročiti'

4001     †gynųti
10550           
10551    †gynųti
Name: base_verb, dtype: object

In [107]:
partial_verb_prefixes.loc[tmp_df.index[:N], "base_noun"] = [
 'pepel', 'stråž', ERR_SYMB + 'bagry', '', 'narod',
   '', 'čar', '', '', '']


In [106]:
N = 10

partial_verb_prefixes.loc[tmp_df.index[:N]].isv.values


array(['izpepeliti', 'nastråžiti', 'obagriti', 'oblačati', 'obnarodovati',
       'obožati', 'očariti', 'očekyvati', 'odračati', 'odročiti'],
      dtype=object)

In [93]:
# vals = [
#        'degradovati', ERR_SYMB + 'niknųti', '', 'sȯsati', ERR_SYMB + 'vihnųti',
#        'naklåti', '', '', '', '',
#        'zrěti', '', '', '', '',
#        '', '', '', '', '',
#        '', '', ERR_SYMB + 'gynųti', '', '',
#        '', 'prostirati', '', '',
#        'klåti']
# partial_verb_prefixes.loc[tmp_df.index[:len(vals)], "base_verb"] = vals



In [64]:


partial_verb_prefixes.loc[13015, "base_verb"] = 'žrěti'
partial_verb_prefixes.loc[17210, "base_verb"] = 'ikati'

partial_verb_prefixes.loc[13978, "base_noun"] = 'smråd'
partial_verb_prefixes.loc[17164, "base_noun"] = 'gospodaŕ'

partial_verb_prefixes.loc[[4592, 9061, 11859, 14707], "base_verb"] = "byti"

In [72]:
partial_verb_prefixes.loc[8276, "base_noun"] = "čudo"
partial_verb_prefixes.loc[10676, "base_noun"] = "poljza"
partial_verb_prefixes.loc[15578, "base_noun"] = "čęsť"
partial_verb_prefixes.loc[12665, "base_noun"] = "råzlika"
partial_verb_prefixes.loc[13896, "base_noun"] = "slovjanin"
partial_verb_prefixes.loc[16523, "base_noun"] = "vliv"

partial_verb_prefixes.loc[[16645, 16646], "base_noun"] = "vonj"
partial_verb_prefixes.loc[[9540, 9541], "base_noun"] = "samota"
partial_verb_prefixes.loc[16511, "base_noun"] = "vlasť"

partial_verb_prefixes.loc[12899, "base_noun"] = "svirěpy"
partial_verb_prefixes.loc[7706, "base_noun"] = "nehaj"
partial_verb_prefixes.loc[9195, "base_noun"] = "oh"

partial_verb_prefixes.loc[[17544], 'base_noun'] = 'zavět'
partial_verb_prefixes.loc[[1988], 'base_noun'] = 'dobry'

partial_verb_prefixes.loc[[15864, 17457], 'base_noun'] = 'sȯn'
partial_verb_prefixes.loc[[8415, 8416], 'base_noun'] = 'obman'

partial_verb_prefixes.loc[[14107, 14574], "base_noun"] = ['sonda', 'studija']

partial_verb_prefixes.loc[11115, "base_noun"] = "poza"


In [88]:
from isv_nlp_utils.slovnik import infer_pos
KNOWN_NOUNS = slovnik[slovnik.partOfSpeech.apply(infer_pos) == 'noun'].isv.unique()

In [95]:
for entry in (
    set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())
):
    stem_cand = entry + 'iz'
    tmp_df2 = partial_verb_prefixes.query("left_stem_cand == @stem_cand")
    print(tmp_df2.isv.values, '->', entry)
    partial_verb_prefixes.loc[tmp_df2.index, "base_noun"] = entry



for entry in (
    set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())
):
    
    stem_cand = entry.replace("ija", 'iz')
    tmp_df2 = partial_verb_prefixes.query("left_stem_cand == @stem_cand")
    if entry != "unija":
        print(tmp_df2.isv.values, '->', entry)
        partial_verb_prefixes.loc[tmp_df2.index, "base_noun"] = entry



['profesionalizovati'] -> profesional
['harakterizovati'] -> harakter
['simbolizovati'] -> simbol
['avtorizovati'] -> avtor
['standardizovati'] -> standard
['idealizovati'] -> ideal
['vulkanizovati'] -> vulkan
['kristalizovati'] -> kristal
['avtomatizovati'] -> avtomat
['kanalizovati'] -> kanal
['signalizovati'] -> signal
['organizovati'] -> organ
['kompjuterizovati'] -> kompjuter
['kanonizovati'] -> kanon
['generalizovati'] -> general
['moralizovati'] -> moral
['agonizovati'] -> agonija
['kolonizovati'] -> kolonija
['kategorizovati'] -> kategorija
['harmonizovati'] -> harmonija


C:\Users\79165\AppData\Local\Temp\ipykernel_33200\766467274.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())
C:\Users\79165\AppData\Local\Temp\ipykernel_33200\766467274.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())


In [ ]:
čati odročiti

In [21]:
ERR_SYMB = "†"

In [928]:
tmp_df[CC].tail(37)

,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns
13711,15001,skaržiti,v.refl. ipf.,complain,NaN,skarž,?karžiti?,,,
13794,29624,skubti,v.intr. ipf.,"pluck, pick, jerk, twitch",NaN,skubti,?kubti?,,,
13808,23372,skvrčati,v.intr. ipf.,sizzle,NaN,skvrč,?kvrčati?,,,
13966,21582,směti,v.tr. ipf.,dare,NaN,sm,?měti?,,,
13978,20177,smŕditi,v.intr. ipf.,stink,NaN,smŕd,?mŕditi?,,,
14006,20715,smykati,v.tr. ipf.,close (ranks),NaN,smyk,?mykati?,,,
14014,18853,snědati,v.intr. ipf.,eat breakfast,NaN,sněd,?nědati?,,,
14037,34003,snovati,v.tr. ipf.,warp,NaN,sn,?novati?,,,
14103,20707,sȯmknųti,v.tr. pf.,close (ranks),NaN,sȯmk,?mknųti?,,,
14113,36052,sȯočati,v.refl. ipf.,"face, deal with",NaN,sȯoč,?čati?,,,


In [916]:

for impossible in ['kladati', 'ložiti', 'niziti', 'končiti', 'jati', 'gynųti']:
    TMP_IDX = tmp_df.query(f"right_stem_cand == '?{impossible}?'").index.tolist()
    if not TMP_IDX:
        continue
    if impossible == "měšćati":
        impossible = "městiti"
    if impossible == "končiti":
        impossible = "končati"
        
    partial_verb_prefixes.loc[TMP_IDX, "base_verb"] = "†" + impossible
TMP_IDX, 1 if TMP_IDX else 0

([4001, 10551], 1)

In [889]:
partial_verb_prefixes[partial_verb_prefixes.ru.str.contains("печат")]

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
7527,14741,natipkati,NaN,v.tr. pf.,3,type,yu,M,"напечатать (на клавиатуре), набрать","надрукаваць, набраць",...,"{набрать, напечатать (на клавиатуре)}","{напиша, натракам}",?,?,tipkati,natipkati,natipk,tipkati,,
8908,23610,odpečętati,NaN,v.tr. pf.,2,"print, unseal",ru bg,NaN,отпечатать,"аддрукаваць, надрукаваць, выдрукаваць, адцісну...",...,{отпечатать},"{печат, !печат}",?,?,pečętati,odpečętati,odpečęt,pečętati,,
8910,32536,odpečętyvati,NaN,v.tr. pf.,2,"#print, unseal",ru bg,NaN,отпечатывать,"аддрукоўваць, выдрукоўваць, адціскаць, адбівац...",...,{отпечатывать},"{печат, !печат}",?,?,pečętati,odpečętyvati,odpečęt,?pečętyvati?,,
9083,23611,odtiskati,NaN,v.tr. ipf.,1,print,ru pl cz sl hr,NaN,отпечатывать,аддрукоўваць,...,{отпечатывать},{!печат},?,?,tiskati,odtiskati,odtisk,tiskati,odtisk,
9084,23612,odtisknųti,NaN,v.tr. pf.,1,print,ru pl cz sl hr,NaN,отпечатать,аддрукаваць,...,{отпечатать},{!печат},?,?,tiskati,odtisknųti,odtisk,tisknųti,odtisk,
10008,2358,pečętati,NaN,v.tr. ipf.,2,print,ru bg,NaN,печатать,друкаваць,...,{печатать},{!печат},?,?,pečętati,pečętati,pečęt,pečętati,,|pečętaŕ
15211,15080,tipkati,NaN,v.tr. ipf.,3,type,yu,I,"печатать (на клавиатуре), набирать","друкаваць (на клавіятуры), набіраць",...,"{печатать (на клавиатуре), набирать}",{!тип},?,?,tipkati,tipkati,tipk,tipkati,,
16945,20274,vtiskati,NaN,v.tr. ipf.,1,"imprint, push in",NaN,NaN,"впечатывать, втискивать, вставлять, вталкивать","удрукоўваць, уціскаць, устаўляць, уштурхваць, ...",...,"{вставлять, впечатывать, втискивать, вталкивать}",{натикам},?,?,tiskati,vtiskati,vtisk,tiskati,,
16946,20276,vtisknųti,NaN,v.tr. pf.,1,"imprint, push in",NaN,NaN,"впечатать, втиснуть, вставить, втолкнуть","удрукаваць, уціснуць, уставіць, уштурхнуць, уп...",...,"{втолкнуть, впечатать, вставить, втиснуть}",{натика},?,?,tiskati,vtisknųti,vtisk,tisknųti,,
17913,-37005,råzpečętati,NaN,v.tr. pf.,NaN,unseal,NaN,NaN,распечатать (открыть),распячатаць (адчыніць),...,{распечатать (открыть)},"{отпечатам (отворя), !разпечатам}",råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,


In [69]:
 

tmp_df.query(f"isv.str.endswith('irati')")


,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
12135,21885,prostirati,NaN,v.tr. ipf.,stretch out,NaN,verb,?,?,,prostirati,prostir,?tirati?,,
12136,21886,prostirati,NaN,v.tr. ipf.,set (table),NaN,verb,?,?,,prostirati,prostir,?tirati?,,


In [70]:
tmp_df.query("isv.str.endswith('ųti')")
#gynųti

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
4131,26209,izniknųti,NaN,v.intr. pf.,"disappear, vanish",NaN,verb,?,?,,izniknųti,iznik,?niknųti?,,
4351,26265,izvihnųti,NaN,v.tr. pf.,"dislocate, sprain",NaN,verb,?,?,,izvihnųti,izvih,?ihnųti?,,
8415,5740,obmanųti,NaN,v.tr. pf.,"con, cheat, delude, deceive, hoodwink",NaN,verb,?,?,,obmanųti,obma,?manųti?,,
8416,18009,obmanųti,NaN,v.tr. pf.,fool,NaN,verb,?,?,,obmanųti,obma,?manųti?,,
13793,34595,skubnųti,NaN,v.intr. pf.,"pluck, pick, jerk, twitch",NaN,verb,?,?,,skubnųti,skub,?kubnųti?,,
14103,20707,sȯmknųti,NaN,v.tr. pf.,close (ranks),NaN,verb,sȯ’,mknųti,,sȯmknųti,sȯmk,?mknųti?,,
14386,6303,stanųti,NaN,v.intr. pf.,"stop (intr.), halt, take a position, stand",NaN,verb,?,?,,stanųti,sta,?tanųti?,|stańje,
15864,21615,usnųti,(usne),v.intr. pf.,fall asleep,NaN,verb,?,?,,usnųti,us,?nųti?,,
17457,19412,zasnųti,NaN,v.intr. pf.,fall asleep,NaN,verb,?,?,,zasnųti,zas,?nųti?,,


In [ ]:
tmp_df.query("isv.str.endswith('ųti')")[CC].isv.str.replace('nųti', 'yti')


In [899]:
tmp_df.right_stem_cand.value_counts().head(15)

?gynųti?        2
?birati?        2
?hati?          2
?pěti?          2
?bųde/bųdųt?    1
?čiti?          1
?ědčiti?        1
?t?             1
?trčiti?        1
?tanųti?        1
?tačiti?        1
?pręgati?       1
?mknųti?        1
?čati?          1
?gašati?        1
Name: right_stem_cand, dtype: int64

In [55]:
tmp_df.right_stem_cand.value_counts().head(15)

?tirati?            9
?lěkati?            4
?njati?             3
?amotiti?           2
?nųti?              2
?manųti?            2
?kalati?            2
?hati?              2
?čarovyvati?        2
?ědčiti?            1
?trčiti?            1
?t?                 1
?čęstvovati?        1
?ŕběti?             1
?tandardizovati?    1
Name: right_stem_cand, dtype: int64

In [ ]:
("jati", "iti"), ("jati", "ti"), ("jati", "nųti"), ("protivdějati", "dějati"), ("vzajemodějati", "dějati")

In [ ]:
#partial_verb_prefixes.loc[[12163, 12162], 'base_verb'] = 'prostiti'
#partial_verb_prefixes.loc[[17544], 'base_noun'] = 'zavět'

In [109]:
tmp_df.isv.str[-4:].value_counts()

vati    9
čiti    6
čati    5
jati    3
žiti    3
riti    3
šati    3
titi    3
nųti    3
žati    2
niti    2
sųt     1
běti    1
siti    1
liti    1
lěti    1
sěti    1
dati    1
tati    1
gati    1
jiti    1
kati    1
měti    1
ubti    1
biti    1
těti    1
rati    1
věti    1
nati    1
Name: isv, dtype: int64

In [849]:
tmp_df.isv.str[-7:].value_counts()

tvovati    3
sbirati    2
bmanųti    2
ědomiti    2
damjati    2
          ..
garšati    1
dvisiti    1
nurjati    1
ovivati    1
ętyvati    1
Name: isv, Length: 120, dtype: int64

In [63]:
vc = tmp_df.right_stem_cand.value_counts()

IRREGULARS = vc[vc > 1].index

In [828]:
EN_TMP = tmp_df.en.value_counts()

EN_TMP = EN_TMP[EN_TMP > 1].index


In [831]:
tmp_df.query('en in @EN_TMP')[CC].sort_values(by="en")

,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns
15794,1779,upȯlnomoćiti,v.tr. pf.,"authorise, authorize, empower",NaN,upȯlnomoć,?pȯlnomoćiti?,,,
15793,1419,upȯlnomoćevati,v.tr. ipf.,"authorise, authorize, empower",NaN,upȯlnomoćev,?pȯlnomoćevati?,,,
13933,21581,slyti,v.intr. ipf.,be renowned,NaN,slyti,?lyti?,,,
13931,21580,slynųti,v.intr. ipf.,be renowned,NaN,sly,?lynųti?,,,
9756,34698,otųpjati,v.tr. ipf.,"blunt, stupefy, make dull",NaN,otųp,?tųpjati?,,,
9755,29768,otųpiti,v.tr. pf.,"blunt, stupefy, make dull",NaN,otųp,?tųpiti?,,,
14103,20707,sȯmknųti,v.tr. pf.,close (ranks),NaN,sȯmk,?mknųti?,,,
14006,20715,smykati,v.tr. ipf.,close (ranks),NaN,smyk,?mykati?,,,
13382,15528,samoubiti,v.intr. pf.,commit suicide,NaN,samoub,?amoubiti?,,,
13453,15530,sebesmŕtiti,v.intr. ipf.,commit suicide,NaN,sebesmŕt,?ebesmŕtiti?,,,


In [111]:
tmp_df.query('right_stem_cand in @IRREGULARS').sort_values(by="right_stem_cand")

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
12828,18617,råzraznjati,NaN,v.tr. ipf.,distinguish,NaN,verb,råz’,raznjati,,råzraznjati,råzrazn,?njati?,,
12136,21886,prostirati,NaN,v.tr. ipf.,set (table),NaN,verb,?,?,,prostirati,prostir,?tirati?,,


In [117]:
IDX = partial_verb_prefixes.query('isv.str.endswith("stirati")').index

partial_verb_prefixes.loc[IDX, "base_verb"] = ERR_SYMB + 'strěti'
partial_verb_prefixes.loc[IDX, "base_noun"] = 'prostor'



In [121]:
IDX = partial_verb_prefixes.query('isv.str.endswith("strěti")').index

partial_verb_prefixes.loc[IDX, "base_verb"] = ERR_SYMB + 'strěti'
partial_verb_prefixes.loc[IDX, "base_noun"] = 'prostor'



In [ ]:
'strěti' in partial_verb_prefixes.isv.unique()

In [29]:
# partial_verb_prefixes.loc[[15864, 17457, 9787], 'derived_nouns'] = ''

In [ ]:
žrěti = žirati
žrati

In [58]:
partial_verb_prefixes.loc[[1988], 'base_noun'] = 'dobry'
partial_verb_prefixes.loc[[17537], 'base_noun'] = 'zautraka'
partial_verb_prefixes.loc[[17544], 'base_noun'] = 'zavět'


In [59]:
IDX = partial_verb_prefixes.query("isv.str.startswith('sȯoč')").index
partial_verb_prefixes.loc[IDX, 'base_noun'] = 'oko'
partial_verb_prefixes.loc[IDX]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns,base_noun
14113,36052,sȯočati,NaN,v.refl. ipf.,"face, deal with",NaN,verb,?,?,sȯočiti,"sȯočati sę s, sųočati sę s",sȯoč,?čati?,,oko
14114,36053,sȯočiti,NaN,v.refl. ipf.,"face, deal with",NaN,verb,?,?,sȯočiti,"sȯočiti sę s, sųočiti sę s",sȯoč,?čiti?,,oko


In [784]:
IDX = partial_verb_prefixes.query("en.str.contains('eat') and isv.str.contains('ži?r')")[CC].index
partial_verb_prefixes.loc[IDX, 'base_verb'] = 'žrěti'


In [872]:
tmp_df.query("en in @KNOWN_EN_DICT")[CC + ['ru']].tail(22)
# plåšiti krviti/krvaviti 
# dobry zautraka zavět propasti věděti  prostiti

,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns,ru
9746,29759,otrězviti,v.tr. pf.,sober up,NaN,otrězv,?trězviti?,,,,отрезвить
9747,34674,otrězvjati,v.tr. ipf.,sober up,NaN,otrězv,?trězvjati?,,,,отрезвлять
10520,36041,pogaršati,v.tr. ipf.,"deteriorate, make worse",NaN,pogarš,?garšati?,,,,ухудшать
11371,23357,prědvyšati,v.tr. ipf.,"prefer, like better",NaN,prědvyš,?šati?,,,,предпочитать
11452,15410,prěnebrěgti,v.tr. pf.,neglect,NaN,prěnebrěgti,?brěgti?,,,,"пренебречь, запустить, забросить, не позаботиться"
11830,21476,prispěti,v.intr. pf.,arrive on time,NaN,prisp,?pěti?,,,,"подоспеть, успеть, прибыть вовремя"
11950,36021,probijati,v.refl. ipf.,"break through, force one’s way through sth.",NaN,probi,?bijati?,,,,"пробиваться, прорываться"
12200,21484,protivrěčiti,v.intr. ipf.,"contradict, deny, negate",NaN,protivrěč,?tivrěčiti?,,,,"противоречить, возражать, отрицать, опровергать"
12665,14964,råzlikovati,v.refl. ipf.,differ,NaN,råzlik,?likovati?,,,|råzličje,"различаться, отличаться"
12828,18617,råzraznjati,v.tr. ipf.,distinguish,NaN,råzrazn,?njati?,,,,различать


In [753]:
# ("agati", "ogti"), ("ihati", "hati"), ("tati", "sti"), ("dati", "sti"), ("irati", "reti"), ("rabrjati", "råbriti"), ("vojiti", "vajati"), ("ylati", "lati")

# ("adnjati", "odniti"), ("avjati", "oviti"), ("govati", "žiti"), ("abjati", "obiti"), ("aljnjati", "oljniti"), ("inati", "něti"), ("ljati", "liti"), ("išćati", "istiti"), ("ajati", "ojiti"), ("rjati", "riti"), ("ažati", "ožiti"), ("niti", "něti"), ("apjati", "opiti"), ("nųti", "ati")
tmp_df.query("en in @KNOWN_EN_DICT")[CC + ['en_cognate']].head(70).tail(37)
#("abnjati", "obniti")




,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns,en_cognate
10498,10788,podvisiti,v.tr. pf.,suspend,NaN,podvis,?isiti?,,,,"[(pozastaviti, staviti)]"
10511,10741,podzirati,v.tr. ipf.,suspect,NaN,podzir,?zirati?,,,,"[(podzrěti, zrěti), (podzrěvati, zrěti)]"
10520,36041,pogaršati,v.tr. ipf.,"deteriorate, make worse",NaN,pogarš,?garšati?,,,,"[(pogoršiti, goršiti)]"
11102,32973,pozlaćati,v.tr. ipf.,gild,NaN,pozlać,?zlaćati?,,,,"[(pozlåtiti, zlåtiti), (zlåtiti, zlåtiti)]"
11371,23357,prědvyšati,v.tr. ipf.,"prefer, like better",NaN,prědvyš,?šati?,,,,"[(volěti, )]"
11452,15410,prěnebrěgti,v.tr. pf.,neglect,NaN,prěnebrěgti,?brěgti?,,,,"[(prěnebrěgati, ), (zanedbati, dbati), (zanedb..."
11830,21476,prispěti,v.intr. pf.,arrive on time,NaN,prisp,?pěti?,,,,"[(prispěvati, pěvati)]"
11910,23175,privlåčivati,v.tr. ipf.,attract,NaN,privlåč,?låčivati?,,,,"[(pritęgati, tęgati), (pritęgnųti, tęgati), (p..."
11950,36021,probijati,v.refl. ipf.,"break through, force one’s way through sth.",NaN,probi,?bijati?,,,,"[(probiti, biti), (prodirati, dreti), (prodret..."
12162,36523,prošćati,v.refl. ipf.,"say goodbye, bid farewell, take one’s leave",NaN,prošć,?šćati?,,,,"[(prostiti, prostiti)]"


In [61]:
RR_OLD = [
    ("arjati", "oriti"), ("aljati", "oliti"), ("anjati", "åniti"), ("ramjati", "råmiti"), ("rastati", "råsti"),
    ("yvati", "vati"), ("yvati", "ati"), ("yvati", "yvti"), ("amyvati", "omiti"),
    ("arjati", "oriti"), ("yvati", "yti"), ("đati", "diti"), ("ćati", "titi"),
            ("vati", "ti"), ("ěvati", "iti"), ("igati", "egti"), ("anjati", "oniti"), 
            ("ađati", "oditi"), ("ađati", "åditi"), ("đati", "děti"), ("žđati", "zditi"),
            ("žđati", "hati"), ("irati", "rati"), ("ati", "ti"), ("reti", "irati"), ("iveti", "iti"), ("žati", "ziti"), ("ažati", "åziti"),
        ("ašati", "åsiti"), ("šati", "siti"), ("ęti", "inati"), ("rěti", "irati"),
        ("agati", "ogti"), ("ihati", "hati"), ("tati", "sti"), ("dati", "sti"), ("irati", "reti"), ("rabrjati", "råbriti"), ("vojiti", "vajati"), ("ylati", "lati"),
        ("adnjati", "odniti"), ("avjati", "oviti"), ("govati", "žiti"), ("abjati", "obiti"), ("aljnjati", "oljniti"), ("inati", "něti"), ("ljati", "liti"), 
        ("išćati", "istiti"), ("ajati", "ojiti"), ("rjati", "riti"), ("ažati", "ožiti"), ("niti", "něti"), ("apjati", "opiti"), ("nųti", "ati"),
        ("aršati", "oršiti"),
        ("šćati", "stiti"), ("ćati", "titi"), ("aćati", "åtiti"), ("šćati", "stiti"), 
        ("orniti", "arnjati"), ("reti", "irati"), ("vpihati", "vȯphati"), ("avjati", "åviti"),
("ykati", "knųti"), ("ositi", "ašati"), ("åčivati", "ěkati"), ("åčivati", "ěkti"), ("åčiti", "ěkti"), ("obŕnųti", "vŕnųti"),
("avnjati", "åvniti"), ("abnjati", "obniti"), 
("njati", "niti"), ("bjati", "biti"),       
        ("jati", "iti"), ("jati", "ti"), ("jati", "nųti"), ("protivdějati", "dějati"), ("vzajemodějati", "dějati"), ("yti", "ynųti"),

            ("ćevati", "ćiti"),
        ("sȯočati", "sȯočiti"), ("parjati", "pråti"), ("ariti", "rěti")

]

In [108]:
# for i, row in tmp_df.query("en not in @KNOWN_EN_DICT")[CC + ['en_cognate']].iterrows():
for i, row in tmp_df.iterrows():
    #'orěti'
    for real, repl in [
            ('ačati', 'očiti')
        ]:
        okanje = row.isv.replace(real, repl)
        if okanje != row.isv and morph.word_is_known(okanje):
            found = partial_verb_prefixes.query('isv == @okanje')
            if not len(found):
                continue
            base_verb_maybe = found.base_verb.values[0]
            print(row.isv, okanje, base_verb_maybe)
            if base_verb_maybe:
                partial_verb_prefixes.loc[i, "base_verb"] = base_verb_maybe
            else:
                # if (" ipf." in row.partOfSpeech and " pf." in found.partOfSpeech.values[0]) or (row.isv == "råzgrađati"):
                if (" ipf." in row.partOfSpeech and " pf." in found.partOfSpeech.values[0]):
                    partial_verb_prefixes.loc[found.index, "base_verb"] = okanje
                    partial_verb_prefixes.loc[i, "base_verb"] = okanje
                else:
                    print("???", row.isv, okanje)
                    print(row.partOfSpeech, found.partOfSpeech.values[0])
                    partial_verb_prefixes.loc[found.index, "base_verb"] = okanje
                    partial_verb_prefixes.loc[i, "base_verb"] = okanje

#                  [i, "base_verb"])

odračati odročiti 


In [587]:
??? råzgrađati råzgråditi
??? zamreti zamirati

oslěpjati oslěpiti lěpiti


'odplati'

In [612]:
#IDX = tmp_df.query("isv.str.endswith('ganjati')")[CC + ['en_cognate']].index

#partial_verb_prefixes.loc[IDX, "base_verb"] = 'gnati'

In [544]:
for i, row in tmp_df.query("en in @KNOWN_EN_DICT")[CC + ['en_cognate']].iterrows():
    for repl in ['orěti', "oriti"]:
        okanje = row.isv.replace("arjati", repl).replace("anjati", "onjati")
        candidates = [x for x in row.en_cognate if x[0] == okanje]
        if candidates:
            # print(row.isv, okanje, candidates)
            # print(row.isv, candidates[0][1])
            partial_verb_prefixes.loc[i, "base_verb"] = candidates[0][1]

In [879]:
tmp_df.left_stem_cand.str[-2:].value_counts()

oč    4
tv    3
ěp    3
ik    3
ti    3
     ..
zn    1
st    1
ro    1
ub    1
ęt    1
Name: left_stem_cand, Length: 76, dtype: int64

In [880]:
tmp_df[tmp_df.left_stem_cand.str[-2:] == 'oč']

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns,en_cognate
8941,3295,odpočivati,NaN,v.intr. ipf.,1,"rest, relax",z j,NaN,отдыхать,"адпачываць, спачываць, аддыхацца, перапачываць",...,"{!отпуснете се, отпуснете се}",?,?,,odpočivati,odpoč,?čivati?,,,"[(odpočęti, počęti)]"
9000,21415,odročiti,NaN,v.tr. pf.,1,"delay, postpone, put off",ru pl cz,NaN,"отложить (на время), отсрочить","адкласці (на тэрмін), адтэрмінаваць",...,"{отлагаме, !отлагаме}",?,?,,odročiti,odroč,?ročiti?,,,None
14113,36052,sȯočati,NaN,v.refl. ipf.,2,"face, deal with",yu mk,NaN,"сталкиваться, иметь дело с","сутыкацца, мець справу з",...,"{!лицето, занимаващи се с}",?,?,,"sȯočati sę s, sųočati sę s",sȯoč,?čati?,,,None
14114,36053,sȯočiti,NaN,v.refl. ipf.,2,"face, deal with",yu mk,NaN,"столкнуться, встретиться","сутыкнуцца, сустрэцца",...,"{!лицето, занимаващи се с}",?,?,,"sȯočiti sę s, sųočiti sę s",sȯoč,?čiti?,,,None


In [437]:
partial_verb_prefixes.shape, tmp_df.shape

((4985, 51), (878, 51))

In [443]:
4985 - 878

4107

In [450]:
print(CC)

['id', 'isv', 'addition', 'partOfSpeech', 'type', 'en', 'sameInLanguages', 'genesis', 'base_verb', 'base_noun', 'derived_nouns']


In [520]:
TMP_IDX = list(set(tmp_df.query("genesis == 'I'")[CC + ['en_cognate']].index.tolist() + [14574, 14677, 12092, 10949]))

partial_verb_prefixes.loc[TMP_IDX, "base_verb"] = partial_verb_prefixes.loc[TMP_IDX, "isv"]

In [518]:
partial_verb_prefixes.loc[1875, "base_verb"] = 'infikovati'

In [506]:
partial_verb_prefixes.loc[[14107, 14574], "base_noun"] = ['sonda', 'studija']


In [507]:
partial_verb_prefixes.loc[12092, "derived_nouns"] = "|propaganda"


In [512]:
partial_verb_prefixes.loc[14677, "derived_nouns"] = "|surfovańje|surfist"
partial_verb_prefixes.loc[10949, "derived_nouns"] = "|postulat"
partial_verb_prefixes.loc[11115, "base_noun"] = "|poza"


In [457]:
CC = ['id', 'isv', 'partOfSpeech', 'en', 'genesis', 'left_stem_cand',
       'right_stem_cand', 'base_verb', 'base_noun', 'derived_nouns']


In [454]:
partial_verb_prefixes.columns

Index(['id', 'isv', 'addition', 'partOfSpeech', 'type', 'en',
       'sameInLanguages', 'genesis', 'ru', 'be', 'uk', 'pl', 'cs', 'sk', 'bg',
       'mk', 'sr', 'hr', 'sl', 'cu', 'de', 'nl', 'eo', 'frequency',
       'intelligibility', 'using_example', 'pos', 'sr_set', 'cs_set', 'cu_set',
       'be_set', 'nl_set', 'sl_set', 'hr_set', 'pl_set', 'en_set', 'de_set',
       'uk_set', 'eo_set', 'sk_set', 'mk_set', 'ru_set', 'bg_set', 'prefix',
       'verb_stem', 'base_verb', 'isv_orig', 'left_stem_cand',
       'right_stem_cand', 'base_noun', 'derived_nouns'],
      dtype='object')

In [473]:
KNOWN = partial_verb_prefixes.base_verb.unique()

partial_verb_prefixes.loc[
    tmp_df.query('isv in @KNOWN').index,
    "base_verb"
] = tmp_df.query('isv in @KNOWN').isv


In [486]:
partial_verb_prefixes.query("en.str.contains('eat')")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
867,4049,biti,(bije),v.tr. ipf.,1,"hit, beat",NaN,NaN,бить,"біць, удараць",...,{бить},"{бия, удрям}",?,?,biti,biti,b,biti,,
899,5574,blågoslavjati,NaN,v.tr. ipf.,1,"bless, beatify",NaN,NaN,благословлять,"бласлаўляць, благаслаўляць",...,{благословлять},{благославям},?,?,blågoslavjati,blågoslavjati,blågoslav,blågoslavjati,,
900,5573,blågosloviti,NaN,v.tr. pf.,1,"bless, beatify",NaN,NaN,благословить,блаславіць,...,{благословить},{благославям},?,?,blågosloviti,blågosloviti,blågosl,blågosloviti,,
925,24693,blejati,NaN,v.intr. ipf.,1,bleat,ru be j,NaN,блеять,бляяць,...,{блеять},{блея},?,?,blejati,blejati,ble,blejati,,
2054,17283,dȯhnųti,NaN,v.intr. pf.,1,breathe,NaN,NaN,"вздохнуть, дохну́ть, дыхнуть","уздыхнуць, дыхнуць",...,"{дохну́ть, вздохнуть, дыхнуть}",{дишам},?,?,dȯhnųti,dȯhnųti,dȯh,dȯhnųti,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17319,31528,zamrznųti,NaN,v.intr. pf.,1,"freeze to death, ice over, be frost-bitten, co...",v z j,NaN,"замерзнуть, застыть","замерзнуць, змерзнуць, застыць, устыць, патужэ...",...,"{застыть, замерзнуть}","{премръзна, измръзна}",?,?,mrznųti,zamrznųti,zamrz,mrznųti,,
17537,14193,zautrakati,NaN,v.intr. ipf.,1,"eat breakfast, have breakfrast",NaN,NaN,завтракать,снедаць,...,{завтракать},{закусвам},?,?,,zautrakati,zautrak,?trakati?,,
17544,31583,zavěćati,NaN,v.tr. ipf./pf.,2,"bequeath, leave by will",ru be sh bm,NaN,завещать,"завяшчаць, адпісваць, запавядаць",...,{завещать},"{завещавам, завещая}",?,?,,zavěćati,zavěć,?ěćati?,,
17792,19402,žariti,NaN,v.tr. ipf.,1,"incandesce, heat to a glow",NaN,NaN,"накалять, раскалять, жарить (сильно греть)","напальваць, распальваць, распаляць, пекчы, пра...",...,"{накалять, жарить (сильно греть), раскалять}",{нажежавам},?,?,žariti,žariti,žar,žariti,žar,


harness                                        4
visit                                          4
cover                                          4
meet, encounter                                4
enrich                                         4
                                              ..
interact                                       2
say goodbye, bid farewell, take one’s leave    2
place, situate, deploy, arrange, distribute    2
forget                                         2
brighten, light up                             2
Name: en, Length: 144, dtype: int64

In [461]:
partial_verb_prefixes[CC].sort_values(by="en")

,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns
17211,19378,zajmati,v.refl. ipf.,"#be concerned with, be occupied, be engaged. w...",NaN,zajm,?jmati?,,,
16490,33615,"vladati, vlåděti",v.intr. ipf.,"#be master of, be in possession of",NaN,"vladati, vlåd","?ladati, vlåděti?",,,
7607,16284,nazyvati,v.refl. ipf.,#be named,NaN,naz,?zyvati?,,,
16486,22477,"vladati, vlåděti",v.intr. ipf.,#be proficient in,NaN,"vladati, vlåd","?ladati, vlåděti?",,,
16875,19125,vrųbati,v.tr. pf.,#carve,NaN,vrųb,rųbati,rųbati,,
...,...,...,...,...,...,...,...,...,...,...
17623,23717,zěvati,v.intr. ipf.,yawn,NaN,zěv,zěvati,zěvati,,
4406,26286,jagniti,v.refl. ipf.,yean,NaN,jagn,jagniti,jagniti,,
9052,21945,odstųpiti,v.tr. pf.,"yield, cede",NaN,odstųp,stųpiti,odstųpati,odstųp,|odstųpnik|odstųpnik
9046,21937,odstųpati,v.tr. ipf.,"yield, cede",NaN,odstųp,stųpati,odstųpati,odstųp,|odstųpnik|odstųpnik


In [460]:
partial_verb_prefixes.query('en in @EN_TMP')[CC].sort_values(by="en")

,id,isv,partOfSpeech,en,genesis,left_stem_cand,right_stem_cand,base_verb,base_noun,derived_nouns
7261,23703,nakryti,v.tr. pf.,cover,NaN,nakryti,kryti,kryti,,
7262,23702,nakryvati,v.tr. ipf.,cover,NaN,nakr,?kryvati?,,,
8374,20559,oblagati,v.tr. ipf.,cover,NaN,oblag,?lagati?,,,
8406,20563,obložiti,v.tr. pf.,cover,NaN,oblož,?ložiti?,,,|obložka
10622,1769,pokryti,v.tr. pf.,cover,NaN,pokryti,kryti,kryti,,
10627,158,pokryvati,v.tr. ipf.,cover,NaN,pokr,?kryvati?,,,
16816,21200,vpręgti,v.tr. pf.,harness,NaN,vpręgti,?pręgti?,,,
16815,21199,vpręgati,v.tr. ipf.,harness,NaN,vpręg,?pręgati?,,,
17404,21236,zapręgati,v.tr. ipf.,harness,NaN,zapręg,?pręgati?,,,
17405,21237,zapręgti,v.tr. pf.,harness,NaN,zapręgti,?pręgti?,,,


In [432]:
EN_TMP = tmp_df.en.value_counts().head(4).index

tmp_df.query('en in @EN_TMP')

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
7262,23702,nakryvati,NaN,v.tr. ipf.,1,cover,NaN,NaN,накрывать,накрываць,...,{накрывать},{покривам},?,?,,nakryvati,nakr,?kryvati?,,
7557,22221,navěditi,NaN,v.tr. pf.,1,visit,NaN,NaN,навестить,"наведаць; даведацца; праведаць; водвекі, водве...",...,{навестить},{посещавам},?,?,,navěditi,navěd,?ěditi?,,
7558,22220,navěđati,NaN,v.tr. ipf.,1,visit,NaN,NaN,навещать,"наведваць, даведвацца, наведвацца, праведваць",...,{навещать},{посетя},?,?,,navěđati,navěđ,?ěđati?,,
8374,20559,oblagati,NaN,v.tr. ipf.,1,cover,NaN,NaN,"покрывать, облицовывать","накрываць, засцілаць, крыць, абіваць, абабівац...",...,"{облицовывать, покрывать}",{!на кутията},?,?,,oblagati,oblag,?lagati?,,
8406,20563,obložiti,NaN,v.tr. pf.,1,cover,NaN,NaN,"покрыть, накрыть","накрыць, заслаць, абіць, абабіць, пакрыць, абк...",...,"{накрыть, покрыть}",{!на кутията},?,?,,obložiti,oblož,?ložiti?,,|obložka
10627,158,pokryvati,NaN,v.tr. ipf.,1,cover,NaN,NaN,покрывать,"пакрываць, абкрываць, накрываць, ахутваць",...,{покрывать},{!на кутията},?,?,,pokryvati,pokr,?kryvati?,,
10862,1220,posěćati,NaN,v.tr. ipf.,1,visit,NaN,NaN,посещать,"наведваць, адведваць",...,{посещать},{!посетете},?,?,,posěćati,posěć,?ěćati?,,
10876,1024,posětiti,NaN,v.tr. pf.,1,visit,NaN,NaN,посетить,наведаць,...,{посетить},{!посетете},?,?,,posětiti,posět,?ětiti?,,
14524,36398,strěćati,NaN,v.refl. ipf.,1,"meet, encounter",v cs j,NaN,встречаться,"сустракацца, спаткацца, страчацца",...,{встречаться},"{!се срещат, среща}",?,?,,strěćati sę,strěć,?trěćati?,,
14525,1954,strěćati,NaN,v.tr. ipf.,1,"meet, encounter",v cs j,NaN,встречать,"сустракаць, спатыкаць, страчаць",...,{встречать},"{сблъскат, !да се срещнат}",?,?,,strěćati,strěć,?trěćati?,,


In [423]:
tmp_df[tmp_df.right_stem_cand == '?iti?']

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
11075,3687,povysiti,NaN,v.tr. pf.,1,"increase, heighten",NaN,NaN,"повысить, увеличить, усилить","павялічыць, падвысіць, павысіць, збольшыць, па...",...,"{увеличить, повысить, усилить}","{!за увеличаване, повишаване на}",?,?,,povysiti,povys,?iti?,,
11353,3128,prěduprěditi,NaN,v.tr. pf.,1,"warn, caution, admonish",NaN,NaN,предостеречь,"асцерагчы, перасцерагчы",...,{предостеречь},"{!предупреждават предупреждават, инструктирани}",?,?,,prěduprěditi,prěduprěd,?iti?,,
11354,21047,prěduprěditi,NaN,v.tr. pf.,2,notify in advance,ru pl,NaN,предупредить,"папярэдзіць, перасцерагчы, запабегчы, унікнуць",...,{предупредить},{!ще ви уведомим предварително},?,?,,prěduprěditi,prěduprěd,?iti?,,
11355,21048,prěduprěditi,NaN,v.tr. pf.,2,prevent,ru sh,NaN,предотвратить,"запабегчы, перадухіліць, вымінуць, унікнуць",...,{предотвратить},{!за предотвратяване на},?,?,,prěduprěditi,prěduprěd,?iti?,,
11615,22729,prěvysiti,NaN,v.tr. pf.,1,"surpass, exceed",NaN,NaN,"превысить, превзойти","перавысіць, перасягнуць",...,"{превзойти, превысить}","{надвишава, !да победим}",?,?,,prěvysiti,prěvys,?iti?,,
12119,1853,prositi,NaN,v.tr. ipf.,1,"ask, require, beg",NaN,NaN,просить,"прасіць, пытаць, шапкаваць",...,{просить},"{!питам, се изисква писмено становище}",?,?,,prositi,pros,?iti?,,
16443,5627,viti,(vije),v.refl. ipf.,1,"wriggle, curl",NaN,NaN,"виться, извиваться","віцца, круціцца, плесціся, выгінацца, павівацца",...,"{извиваться, виться}","{!кривляться, къдря}",?,?,,viti sę,v,?iti?,,
16444,31233,viti,(vije),v.tr. ipf.,1,twist,v j,NaN,"вить, свивать, скручивать","віць, звіваць, павіваць",...,"{свивать, скручивать, вить}",{!обрат},?,?,,viti,v,?iti?,,
16445,33571,viti,(vije),v.tr. ipf.,1,#twine,ru uk pl cz bg,NaN,"оплетать, обвивать","аплятаць, абвіваць, апавінаць, увінаць",...,"{обвивать, оплетать}",{!влакното},?,?,,viti,v,?iti?,,
16446,33572,viti,(vije),v.tr. ipf.,1,wind,ru uk cz sl bm,NaN,вить (гнездо),віць (гняздо),...,{вить (гнездо)},{!вятър},?,?,,viti,v,?iti?,,


In [421]:
tmp_df.query("derived_nouns != ''")[COLS].isv.unique()

array(['definiovati', 'degustovati', 'demonstrovati', 'deportovati',
       'dobrěti', 'doskonaliti', 'izolovati', 'izvěstiti',
       'nacionalizovati', 'naložiti', 'narěkati', 'nevtralizovati',
       'obložiti', 'observovati', 'orati', 'orientovati', 'orųdovati',
       'orųžiti', 'ostriti', 'ovivati', 'ozdråviti', 'poražati',
       'posylati', 'prěčiti', 'prědložiti', 'primětiti', 'produkovati',
       'profanovati', 'prostiti', 'protiviti', 'råzlikovati', 'råzsylati',
       'sbirati', 'simulovati', 'sinhronizovati', 'sirotěti', 'skopiti',
       'skųpiti', 'slaběti', 'slaviti', 'slěpnųti', 'slovjanizovati',
       'smrkati', 'snižati', 'sȯsati', 'spekulovati', 'stabilizovati',
       'stanoviti', 'stanųti', 'starati', 'starěti', 'stavati', 'staviti',
       'stavjati', 'sterilizovati', 'stråniti', 'strěliti', 'strěljati',
       'studiti', 'sunųti', 'sųt', 'usnųti', 'variovati', 'važiti',
       'ventilovati', 'verifikovati', 'věriti', 'veseliti', 'vibrovati',
       'vlastniti'

In [422]:
tmp_df.query("isv.str.endswith('rěkati')")[COLS]

,id,isv,addition,partOfSpeech,en,genesis,pos,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,derived_nouns
4234,21303,izrěkati,NaN,v.tr. ipf.,"say, utter",NaN,verb,?,?,,izrěkati,izrěk,?rěkati?,
7388,21354,narěkati,NaN,v.intr. ipf.,"complain, lament",NaN,verb,?,?,,narěkati,narěk,?rěkati?,|narěčje|narěčje
7389,33141,narěkati,NaN,v.tr. ipf.,give a name,NaN,verb,?,?,,narěkati,narěk,?rěkati?,|narěčje|narěčje
8993,21404,odrěkati,NaN,v.refl. ipf.,"give up, renounce, repudiate, disown",NaN,verb,?,?,,odrěkati sę,odrěk,?rěkati?,
8994,21406,odrěkati,NaN,v.tr. ipf.,"cancel, annul, deny",NaN,verb,?,?,,odrěkati,odrěk,?rěkati?,
10473,5612,podstrěkati,(podstrěče),v.tr. ipf.,"incite, instigate, abet, stir up, rouse",NaN,verb,?,?,,podstrěkati,podstrěk,?trěkati?,
11498,21465,prěrěkati,NaN,v.refl. ipf.,#make a slip of the tongue,NaN,verb,?,?,,prěrěkati sę,prěrěk,?rěkati?,
15827,21603,urěkati,NaN,v.tr. ipf.,"bewitch, cast a spell over",NaN,verb,?,?,,urěkati,urěk,?rěkati?,
17426,21665,zarěkati,NaN,v.refl. ipf.,"pledge, vow",NaN,verb,?,?,,zarěkati sę,zarěk,?rěkati?,


In [416]:
partial_verb_prefixes


,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
5,11,abdikovati,NaN,v.intr. ipf.,1,abdicate,NaN,I,отрекаться,адракацца,...,{отрекаться},{абдикирам},?,?,abdikovati,abdikovati,abdik,abdikovati,,|abdikacija
16,6119,abonovati,NaN,v.tr. ipf.,1,"subscribe, engage",NaN,F,"подписываться, абонировать","падпісвацца, абаніраваць",...,"{подписываться, абонировать}",{абонирам},?,?,abonovati,abonovati,abon,abonovati,,
28,19625,absorbovati,NaN,v.tr. ipf.,2,absorb,NaN,I,"абсорбировать, поглощать","абсарбаваць, паглынаць",...,"{абсорбировать, поглощать}",{абсорбирам},?,?,absorbovati,absorbovati,absorb,absorbovati,,|absorbcija
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,1,abstract,NaN,I,абстрагировать,абстрагаваць,...,{абстрагировать},{абстрахирам се от},?,?,abstrahovati,abstrahovati,abstrah,abstrahovati,,
43,24070,adaptovati,NaN,v.tr. ipf./pf.,1,adapt,NaN,I,адаптировать,"адаптаваць, прыстасаваць",...,{адаптировать},"{пригаждам, адаптирам}",?,?,adaptovati,adaptovati,adapt,adaptovati,,|adaptacija
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473,žužati,NaN,v.intr. ipf.,1,buzz,NaN,NaN,жужжать,"гусці, гудзець",...,{жужжать},{жужа},?,?,žužati,žužati,žuž,žužati,,
17913,-37005,råzpečętati,NaN,v.tr. pf.,NaN,unseal,NaN,NaN,распечатать (открыть),распячатаць (адчыніць),...,{распечатать (открыть)},"{отпечатам (отворя), !разпечатам}",råz’,pečętati,pečętati,råzpečętati,råzpečęt,pečętati,,
17914,-37006,råzpečętyvati,NaN,v.tr. ipf.,NaN,unseal,NaN,NaN,распечатывать (открывать),распячатваць (адчыніць),...,{распечатывать (открывать)},"{отпечатвам (отварям), !разпечатвам}",råz’,pečętyvati,pečętati,råzpečętyvati,råzpečęt,?pečętyvati?,,
17917,-30731,smogti,NaN,v.tr. pf.,NaN,"manage to, be able to, cope, make it",NaN,NaN,"смочь, суметь","змагчы, здолець",...,"{смочь, суметь}",{!смогвам},?,?,mogti,smogti,smogti,mogti,,


In [414]:
tmp_df.query("left_stem_cand.str.endswith('av') and derived_nouns == ''")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
4256,20398,izslavjati,NaN,v.refl. ipf.,1,express oneself verbally,NaN,NaN,выражаться,"выказвацца, выяўляцца",...,{выражаться},{изразявам се},?,?,,izslavjati sę,izslav,?lavjati?,,
4257,20397,izslavjati,NaN,v.tr. ipf.,1,put into words,NaN,NaN,"выражать, излагать, формулировать","выказваць, фармуляваць",...,"{излагать, выражать, формулировать}","{изразявам, изказвам, изричам}",?,?,,izslavjati,izslav,?lavjati?,,
7595,32962,nazdravjati,NaN,v.intr. ipf.,2,"toast, drink a toast",j,NaN,"поднимать тост, выпивать тост","паднімаць тост, падымаць тост, выпіваць тост",...,"{поднимать тост, выпивать тост}","{вдигам наздравица, вдигам тост}",?,?,,nazdravjati,nazdrav,?zdravjati?,,
8439,151,obnavjati,NaN,v.tr. ipf.,1,"renew, refurbish",NaN,NaN,обновлять,"абнаўляць, паднаўляць, зажываць",...,{обновлять},"{поднови, !ъпгрейд}",?,?,,obnavjati,obnav,?jati?,,
8440,34417,obnavjati,NaN,v.tr. ipf.,1,restore,z j,NaN,"реставрировать, ремонтировать","рэстаўраваць, рэстаўрыраваць",...,"{реставрировать, ремонтировать}",{!възстановяване},?,?,,obnavjati,obnav,?jati?,,
8592,1090,obstanavjati,NaN,v.refl. ipf.,1,stop,NaN,NaN,останавливаться,"спыняцца, запінацца, запыняцца, застанаўлівацц...",...,{останавливаться},{спирам се},?,?,,obstanavjati sę,obstanav,?tanavjati?,,
8593,34789,obstanavjati,NaN,v.tr. ipf.,1,stop,NaN,NaN,останавливать,"спыняць, затрымліваць, сунімаць, прыпыняць",...,{останавливать},{спирам},?,?,,obstanavjati,obstanav,?tanavjati?,,
9808,10401,ozdravjati,NaN,v.tr. ipf.,1,cure,v z j,NaN,"оздоровлять, оздоравливать, лечить","аздараўляць, саніраваць, лячыць, лекаваць",...,"{оздоравливать, лечить, оздоровлять}","{лекувам, оздравявам}",?,?,,ozdravjati,ozdrav,?zdravjati?,,
10922,21745,postanavjati,NaN,v.tr. ipf.,2,"decide, decree",ru pl,NaN,"постановлять, решать","пастанаўляць, вырашаць, меркаваць",...,"{постановлять, решать}",{!решение декрет},?,?,,postanavjati,postanav,?tanavjati?,,
11110,6323,poznavati,NaN,v.tr. ipf.,1,"recognise, recognize",NaN,NaN,признавать,прызнаваць,...,{признавать},"{да признае, !да се признае}",?,?,,poznavati,poznav,?znavati?,,


In [400]:
tmp_df.query("left_stem_cand.str.endswith('ać') and derived_nouns == ''")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
4166,8087,izplaćati,NaN,v.tr. ipf.,1,pay out,v z j,NaN,выплачивать,"выплачваць, выплочваць",...,{выплачивать},{изплащам},?,?,,izplaćati,izplać,?plaćati?,,
8321,17990,obhvaćati,NaN,v.tr. ipf.,1,"embrace, clasp",NaN,NaN,"обхватывать, обнимать","абхапляць, абхопліваць, абхватваць, абнімаць, ...",...,"{обхватывать, обнимать}","{обхватить, !прегръдка}",?,?,,obhvaćati,obhvać,?hvaćati?,,
8462,242,obogaćati,NaN,v.tr. ipf.,1,enrich,NaN,NaN,обогащать,"абагачаць, узбагачаць, багаціць, збагаціць",...,{обогащать},{!обогатяване на},?,?,,obogaćati,obogać,?gaćati?,,
8485,22682,obraćati,(+3),v.refl. ipf.,1,"address, appeal to",ru cs yu mk,NaN,обращаться к,"звяртацца да, зварочвацца да",...,{обращаться к},{!адрес обжалва},?,?,,obraćati sę k,obrać,?raćati?,,
8486,22674,obraćati,NaN,v.refl. ipf.,1,"turn (intr.), turn around",NaN,NaN,"обращаться, поворачиваться, вращаться, оборачи...","абарочвацца, абарачацца, наварачацца, паварочв...",...,"{поворачиваться, вращаться, обращаться, оборач...","{!на завой (интр.), обърни се}",?,?,,obraćati sę,obrać,?raćati?,,
8487,22676,obraćati,NaN,v.refl. ipf.,1,turn (into sth.),ru pl cz,NaN,"превращаться, оборачиваться","ператварацца, пераварочвацца, абарачацца",...,"{превращаться, оборачиваться}",{!на завой (в n.)},?,?,,obraćati sę,obrać,?raćati?,,
8488,22678,obraćati,NaN,v.refl. ipf.,1,convert (to a religion),ru cz sh,NaN,"обращаться, переходить (в религию)","навярняцца, пераходзіць (у рэлігію)",...,"{переходить (в религию), обращаться}",{!преобразуване (религия)},?,?,,obraćati sę,obrać,?raćati?,,
8489,3206,obraćati,NaN,v.tr. ipf.,1,turn around,NaN,NaN,"поворачивать, оборачивать","паварочваць, абарачаць",...,"{поворачивать, оборачивать}",{!обърни се},?,?,,obraćati,obrać,?raćati?,,
8490,22665,obraćati,NaN,v.tr. ipf.,2,"rotate, revolve (tr.)",uk pl yu,NaN,"вращать, крутить","варочаць, вярцець, круціць",...,"{вращать, крутить}","{!върти, върти (tr.)}",?,?,,obraćati,obrać,?raćati?,,
8491,22667,obraćati,NaN,v.tr. ipf.,1,convert (sth. into sth.),ru pl cz,NaN,"превращать, обращать","ператвараць, пераварочваць",...,"{превращать, обращать}",{!преобразуване (СТХ. в СТГ.)},?,?,,obraćati,obrać,?raćati?,,


In [404]:
partial_verb_prefixes.loc[
    tmp_df.query("left_stem_cand.str.endswith('jdti') and derived_nouns == ''").index,
    "base_verb"
] = 'idti'

In [385]:
for entry in (
    set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())
):
    stem_cand = entry + 'iz'
    tmp_df = partial_verb_prefixes.query("left_stem_cand == @stem_cand")
    print(tmp_df.isv.values, '->', entry)
    partial_verb_prefixes.loc[tmp_df.index, "base_noun"] = entry

['kanalizovati'] -> kanal
['simbolizovati'] -> simbol
['profesionalizovati'] -> profesional
['idealizovati'] -> ideal
['harakterizovati'] -> harakter
['kompjuterizovati'] -> kompjuter
['standardizovati'] -> standard
['avtomatizovati'] -> avtomat
['kristalizovati'] -> kristal
['avtorizovati'] -> avtor
['kanonizovati'] -> kanon
['moralizovati'] -> moral
['generalizovati'] -> general
['signalizovati'] -> signal
['vulkanizovati'] -> vulkan
['organizovati'] -> organ


C:\Users\79165\AppData\Local\Temp\ipykernel_16780\4070380472.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())


In [384]:

for entry in (
    set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())
):
    stem_cand = entry.replace("ija", 'iz')
    tmp_df = partial_verb_prefixes.query("left_stem_cand == @stem_cand")
    print(tmp_df.isv.values, '->', entry)
    partial_verb_prefixes.loc[tmp_df.index, "base_noun"] = entry

['agonizovati'] -> agonija
['harmonizovati'] -> harmonija
['kategorizovati'] -> kategorija
['uniziti'] -> unija
['kolonizovati'] -> kolonija


C:\Users\79165\AppData\Local\Temp\ipykernel_16780\4281741.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())


In [381]:
    set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())


C:\Users\79165\AppData\Local\Temp\ipykernel_16780\3739348245.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  set(partial_verb_prefixes.left_stem_cand.str.replace("iz$", "ija").unique()) & set(KNOWN_NOUNS) - set(partial_verb_prefixes.left_stem_cand.unique())


{'agonija', 'harmonija', 'kategorija', 'kolonija', 'unija'}

In [380]:
entry

'kolonija'

In [379]:
partial_verb_prefixes.query("isv.str.contains('agon')")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
99,24121,agonizovati,NaN,v.intr. ipf.,NaN,"agonise, agonize",NaN,I,"агонизировать, мучиться","аганізаваць, мучыцца",...,"{мучиться, агонизировать}",{агонизирам},?,?,agonizovati,agonizovati,agoniz,agonizovati,,


In [359]:
partial_verb_prefixes.query("base_verb == '' and base_noun == '' and derived_nouns != ''")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun,derived_nouns
1189,34791,buditi sę,NaN,v.tr. ipf.,1,wake up,NaN,NaN,"просыпаться, пробуждаться","прачынацца, абуджацца",...,"{просыпаться, пробуждаться}",{събудя се},?,?,,buditi sę,bud,?buditi sę?,,|budka
1729,4956,definiovati,NaN,v.tr. ipf.,1,define,NaN,I,определять,вызначаць,...,{определять},"{определям, дефинирам}",’,definiovati,,definiovati,defini,?finiovati?,,|definicija
1744,25175,degustovati,NaN,v.tr. ipf./pf.,2,taste,NaN,I,дегустировать,дэгуставаць,...,{дегустировать},"{дегустирам, вкусвам}",’,degustovati,,degustovati,degust,?gustovati?,,|degustacija
1795,6266,demonstrovati,NaN,v.tr. ipf.,1,demonstrate,NaN,I,демонстрировать,дэманстраваць,...,{демонстрировать},"{демонстрирам, показвам}",’,demonstrovati,,demonstrovati,demonstr,?monstrovati?,,|demonstracija
1806,398,deportovati,NaN,v.tr. ipf./pf.,1,deport,NaN,I,депортировать,дэпартаваць,...,{депортировать},{депортирам},de’,portovati,,deportovati,deport,?rtovati?,,|deportacija
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16871,22641,vŕtěti,(vŕti),v.tr. ipf.,2,"drill, bore",pl sh,NaN,"сверлить, бурить",свідраваць,...,"{сверлить, бурить}",{дупча},?,?,,vŕtěti,vŕt,?ŕtěti?,,|vŕtka
16872,22643,vŕtěti,(vŕti),v.intr. ipf.,2,wag,pl cz,NaN,"махать, качать","махаць, хістаць",...,"{махать, качать}",{махам},?,?,,vŕtěti,vŕt,?ŕtěti?,,|vŕtka
16974,31404,vulkanizovati,NaN,v.tr. ipf./pf.,1,"vulcanise, vulcanize",NaN,I,вулканизировать,вуканізаваць,...,{вулканизировать},"{вулканизира, вулканизирам}",?,?,,vulkanizovati,vulkaniz,?lkanizovati?,,|vulkanizacija
17277,1740,založiti,NaN,v.tr. pf.,2,"establish, set up, found",NaN,NaN,"учредить, основать, создать","заснаваць, зафундаваць, стварыць",...,"{учредить, создать, основать}","{учредя, основа}",?,?,,založiti,založ,?ložiti?,,|založnik


In [ ]:
izvoliti -> liti
nezadovoliti -> liti
obvodniti -> dniti
osoliti -> liti
osušiti -> šiti
privoliti -> liti
udariti -> dariti
zadovoliti -> liti

žiti
byti
byvati


råzględěti -> ględěti / ględati



In [ ]:
spekulacija stabilizacija

In [159]:
noun_stem_cand = partial_verb_prefixes.left_stem_cand.apply(lambda x: bite_all_prefixes_off(x, "")[0])

fill_nouns = noun_stem_cand[noun_stem_cand.apply(lambda x: x in KNOWN_NOUNS)]

partial_verb_prefixes.loc[fill_nouns.index, "base_noun"] = fill_nouns

In [241]:
noun_stem_cand = partial_verb_prefixes.left_stem_cand

fill_nouns2 = noun_stem_cand[(noun_stem_cand.apply(lambda x: x in KNOWN_NOUNS)) & (partial_verb_prefixes.base_noun == '')]

# new_fill = (set(fill_nouns2.index) - set(fill_nouns.index))
len(fill_nouns2)

239

In [242]:
partial_verb_prefixes.loc[fill_nouns2.index, "base_noun"] = fill_nouns2

In [ ]:
partial_verb_prefixes.loc[fill_nouns2.index, "base_noun"] = fill_nouns2

In [219]:
'debjut' in KNOWN_NOUNS

True

In [169]:
slovnik[slovnik.isv.str.endswith("acija")]

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,sr_set,hr_set,nl_set,en_set,de_set,bg_set,sl_set,ru_set,eo_set,cu_set
4,35658,abdikacija,NaN,f.,1,abdication,NaN,I,"отречение, абдикация, сложение полномочий","абдыкацыя, вырачэнне, адрачэнне, адпінанне, вы...",...,{одрицање},{abdikacija},"{aftreden, abdicatie}",{abdication},{Abdankung},{абдикация},{odstop},"{отречение, сложение полномочий, абдикация}",{abdiko},{отреченїе}
41,24068,adaptacija,NaN,f.,1,adaptation,NaN,I,адаптация,"адаптацыя, прыстасаванасць",...,{адаптација},{adaptacija},"{adaptatie, aanpassing}",{adaptation},{Anpassung},{адаптация},"{adaptacija, prilagoditev}",{адаптация},{adapto},{!}
47,2848,administracija,NaN,f.,1,administration,NaN,I,администрация,адміністрацыя,...,"{администрација, управа}","{administracija, uprava}",{administratie},{administration},{Verwaltung},{администрация},"{administracija, uprava}",{администрация},{administrado},{!}
89,24111,agitacija,NaN,f.sg.,1,agitation,NaN,I,агитация,агітацыя,...,{агитација},{agitacija},{agitatie},{agitation},{Agitation},{агитация},{agitacija},{агитация},{agitado},{!}
93,35785,aglomeracija,NaN,f.,1,agglomeration,v z j,I,агломерация,агламерацыя,...,"{нагомилавање, агломерација}",{nagomilavanje},{agglomeratie},{agglomeration},"{Metropolregion, Agglomeration}",{агломерация},"{kopičenje, !aglomeracija}",{агломерация},{aglomeraĵo},{!}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16262,32836,verifikacija,NaN,f.,1,verification,NaN,I,"верификация, проверка","верыфікацыя, праверка, спраўджванне",...,"{верификација, провера}","{verifikacija, ovjera}",{verificatie},{verification},{!Nachprüfung},"{заверка, верификация}",{!Ta preverjanja},"{проверка, верификация}",{!konfirmo},{!}
16355,31178,vibracija,NaN,f.,1,vibration,NaN,I,вибрация,вібрацыя,...,"{трептај, вибрација, њихање}",{vibracija},"{vibratie, trilling}",{vibration},{!Vibration},{вибрация},"{bliskavico, !vibracije, nihanje}",{вибрация},{!vibrado},{!}
16593,35493,vojenna aviacija,NaN,f.,1,airforce,NaN,NaN,военная авиация,"вайсковая авіяцыя, ваенная авіяцыя",...,{ратно ваздухопловство},"{vojno zrakoplovstvo, zračna flota, vojna avij...",{luchtmacht},{airforce},{!Luftwaffe},"{военна авиация, военно въздухоплаване}",{!zračne sile},{военная авиация},{!aerarmeon},{!}
16967,31395,vulgarizacija,NaN,f.,1,"vulgarisation, vulgarization",NaN,I,вульгаризация,вульгарызацыя,...,"{упрошћавање, вулгаризација}",{vulgarizacija},{vulgarisering},"{vulgarization, vulgarisation}","{!vulgarisation, Vulgarisierung}",{опростачване},"{poenostavitev, !vulgarization}",{вульгаризация},"{!vulgarisation, vulgarización}",{!}


In [173]:
slovnik[(slovnik.isv.str.endswith("ija")) & (1 - slovnik.isv.str.endswith("cija"))]

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,sr_set,hr_set,nl_set,en_set,de_set,bg_set,sl_set,ru_set,eo_set,cu_set
8,6165,abhazija,NaN,f.sg.,1,Abkhazia,NaN,NaN,Абхазия,Абхазія,...,{Абхазија},{Abhazija},{Abchazië},{Abkhazia},{Abchasien},{Абхазия},{Abhazija},{Абхазия},{Abĥazio},{Абхаꙁі́ꙗ}
64,36566,adžarija,NaN,f.sg.,1,"Adjara, Adzharia",v z j,NaN,Аджария,"Аджара, Аджарыя",...,{Аџарија},{Adžarija},{Adzjarië},"{Adjara, Adzharia}",{Adscharien},{Аджария},{Adžarija},{Аджария},{Aĝario},{!}
72,19629,afazija,NaN,f.sg.,1,aphasia,NaN,I,афазия,афазія,...,{афазија},{afazija},{afasie},{aphasia},{Aphasie},{афазия},{afazija},{афазия},{afazio},{!}
98,24120,agonija,NaN,f.sg.,NaN,agony,NaN,I,агония,агонія,...,{агонија},{agonija},{ondraaglijke pijn},{agony},{Qual},{агония},{agonija},{агония},{agonio},{!}
102,24127,agresija,NaN,f.sg.,1,aggression,NaN,I,агрессия,агрэсія,...,{агресија},"{agresija, napad}",{agressie},{aggression},{Aggression},{агресия},"{agresija, napad}",{агрессия},"{agreso, atakemo}",{!}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16457,23075,vizija,NaN,f.,1,#vision,NaN,I,ви́дение,"візія, відзеш, відмо, бачанне",...,{визија},{!vizija},{visie},{#vision},{!Vision},"{изглед, визия}",{!vizija},{ви́дение},{!#vision},{!}
16692,13779,vȯzdušna divizija,NaN,f.,1,airborne division,NaN,NaN,воздушно-десантная дивизия,паветрана-дэсантная дывізія,...,{ваздушна дивизија},{!vazdušna divizija},{luchtlandingsdivisie},{airborne division},{!airborne Division},{въздушнодесантна дивизия},{!delitev zraka},{воздушно-десантная дивизия},{!aero divido},{!}
16965,2430,vųgrija,NaN,f.sg.,1,Hungary,NaN,NaN,Венгрия,"Вугоршчына, Венгрыя",...,{Мађарска},{Mađarica},{Hongarije},{Hungary},{!Ungarn},"{Маджарско, Унгария}",{!Madžarska},{Венгрия},{!Hungario},{!}
17282,36510,zambija,NaN,f.,1,Zambia,v z j,NaN,Замбия,Замбія,...,{Замбија},{Zambija},{Zambia},{Zambia},{!Sambia},{Замбия},{!Zambija},{Замбия},{!Zambio},{!}


In [165]:
partial_verb_prefixes.loc[fill_nouns.index].query('base_verb == ""')

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,mk_set,ru_set,bg_set,prefix,verb_stem,base_verb,isv_orig,left_stem_cand,right_stem_cand,base_noun
1742,25172,degradovati,NaN,v.tr. ipf./pf.,1,"degrade, demote",NaN,I,деградировать,дэградаваць,...,{деградира},{деградировать},"{унижавам, деградирам, понижавам}",’,degradovati,,degradovati,degrad,?gradovati?,grad
4061,3603,izkusiti,NaN,v.tr. pf.,1,"experience, undergo",NaN,NaN,"испытать, испробовать, претерпеть, перенести","выпрабаваць, праверыць",...,"{искуси, проба}","{претерпеть, испытать, перенести, испробовать}",{изпитам},?,?,,izkusiti,izkus,?kusiti?,kus
4102,17568,izmirati,NaN,v.intr. ipf.,1,go extinct,NaN,NaN,вымирать,выміраць,...,{изумира},{вымирать},{измирам},?,?,,izmirati,izmir,?mirati?,mir
4159,26220,izpepeliti,NaN,v.tr. pf.,1,burn to ashes,NaN,NaN,"испепелить, сжечь дотла","спапяліць, спаліць дашчэнту",...,"{прегори, пепелоса}","{испепелить, сжечь дотла}",{изпепеля},?,?,,izpepeliti,izpepel,?pepeliti?,pepel
4308,2393,iztirati,NaN,v.tr. ipf.,1,erase,NaN,NaN,"затирать, стирать","выціраць, заціраць, сціраць",...,"{истрие, избрише}","{стирать, затирать}","{изтривам, заличавам, изтърквам}",?,?,,iztirati,iztir,?tirati?,tir
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17182,571,zahvatiti,NaN,v.tr. pf.,1,capture,NaN,NaN,захватить,"захапіць, апантаць",...,"{заплени, фати}",{захватить},"{хвана, запленя}",?,?,,zahvatiti,zahvat,?hvatiti?,hvat
17230,31508,zaklinati,NaN,v.tr. ipf.,1,"charm, conjure, bewitch, incant",v z j,NaN,"заклинать, околдовывать","заклінаць, замаўляць, зачароўваць",...,"{шармира, маѓепсува, опседнува}","{заклинать, околдовывать}","{омагьосам, очаровам}",?,?,,zaklinati,zaklin,?klinati?,klin
17231,31509,zakliniti,NaN,v.tr. pf.,1,wedge,v z bg,NaN,"заклинить, вклинить","заклінаваць, уклінаваць",...,"{закова, стави клин}","{вклинить, заклинить}",{вклиня},?,?,,zakliniti,zaklin,?kliniti?,klin
17232,33665,zaklinovati,NaN,v.tr. ipf.,1,wedge,v z bg,NaN,"заклинивать, клинить, вклинивать","закліноўваць, клінаваць, укліноўваць, укліньваць",...,"{кова, става клин}","{вклинивать, заклинивать, клинить}",{вклинявам},?,?,,zaklinovati,zaklin,?klinovati?,klin


In [154]:
KNOWN_NOUNS = slovnik[slovnik.partOfSpeech.apply(infer_pos) == 'noun'].isv.unique()

In [151]:
from isv_nlp_utils.slovnik import infer_pos

In [312]:
cnt
    


Counter({1: 2721, 2: 507, 3: 33})

In [314]:
cnt2

Counter({2: 120, 3: 7})

In [310]:
partial_verb_prefixes.query("base_verb == 'obobćati'")


,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand,base_verb,isv_orig
8458,35425,obobćati,NaN,v.tr. ipf.,1,"generalise, generalize",ru cs sh bm,NaN,обобщать,"абагульняць, падагульняць",...,"{!komunigi, ĝeneraligi}",{zovšeobecňovať},{обопштува генерализира},{обобщать},"{!домысливать, обобщава}",?,?,obobć,obobćati,obobćati
8460,35426,obobćiti,NaN,v.tr. pf.,1,"generalise, generalize",ru cs sh bm,NaN,обобщить,"абагульніць, абагуліць",...,"{!komunigi, ĝeneraligi}",{zovšeobecniť},"{обопшти, генерализира}",{обобщить},"{!домысливать, обобщава}",?,?,obobć,obobćati,obobćiti


In [315]:
mbs

[('b',
  array(['biti', 'byvati'], dtype=object),
  array(['biti', ''], dtype=object)),
 ('cěl',
  array(['cěliti', 'cělovati'], dtype=object),
  array(['cěliti', ''], dtype=object)),
 ('dar',
  array(['dariti', 'dariti', 'darovati', 'darovati'], dtype=object),
  array(['dariti', 'dariti', '', ''], dtype=object)),
 ('izl',
  array(['izliti', 'izlivati', 'izloviti'], dtype=object),
  array(['liti', '', ''], dtype=object)),
 ('k',
  array(['kovati', 'kyvati'], dtype=object),
  array(['kovati', ''], dtype=object)),
 ('kaz',
  array(['kazati sę', 'kazati', 'kazati', 'kaziti'], dtype=object),
  array(['kazati', 'kazati', 'kazati', ''], dtype=object)),
 ('l',
  array(['liti', 'liti', 'loviti', 'loviti'], dtype=object),
  array(['liti', 'liti', '', ''], dtype=object)),
 ('nab',
  array(['nabiti', 'nabivati', 'nabyvati'], dtype=object),
  array(['biti', '', ''], dtype=object)),
 ('natęg',
  array(['natęgati', 'natęgnųti'], dtype=object),
  array(['tęgati', ''], dtype=object)),
 ('obruš',
  arr

In [62]:
KNOWN_BASES

NameError: name 'KNOWN_BASES' is not defined

In [245]:
KNOWN_BASES = partial_verb_prefixes.query("base_verb != ''").base_verb.unique()

KNOWN_BASES

array(['abdikovati', 'abonovati', 'absorbovati', ..., 'žalovati',
       'žariti', 'žužati'], dtype=object)

In [246]:
for i, row in partial_verb_prefixes.query("base_verb == ''").iterrows():
    x = row.isv
    cand, pref = bite_all_prefixes_off(x, "")
    if cand in KNOWN_BASES:
        if cand == x:
            print(x, "==", cand)
            # write_base_verb([i], cand)
        else:
            POS_cands = partial_verb_prefixes.query("isv == @cand")
            if " pf" in row.partOfSpeech or ".pf" in row.partOfSpeech:
                if POS_cands.partOfSpeech.str.contains("[ .]" + "ipf").any():
                    # write_base_verb([i], cand)
                    print(x, "->", cand)
                else:
                    pass
                    print(x, "->", cand)
                    print(row.partOfSpeech)
                    print(POS_cands.partOfSpeech.values, POS_cands.isv_orig.values)
                    #write_base_verb([i], cand)

            else:
                pass
                print("???", x, "???", cand)
                print(row.partOfSpeech)
                print(POS_cands.partOfSpeech.values, POS_cands.isv_orig.values)
                #write_base_verb([i] + POS_cands.index.tolist(), cand)

                print()


In [294]:
partial_verb_prefixes.query("base_verb == ''")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand,base_verb,isv_orig
5,11,abdikovati,NaN,v.intr. ipf.,1,abdicate,NaN,I,отрекаться,адракацца,...,{abdiki},{abdikovať},"{откажува, абдицира}",{отрекаться},{абдикирам},?,?,abdik,,abdikovati
16,6119,abonovati,NaN,v.tr. ipf.,1,"subscribe, engage",NaN,F,"подписываться, абонировать","падпісвацца, абаніраваць",...,{aboni},"{zapísať sa (na odber), prihlásiť sa, predplat...","{абонира, претплатува}","{подписываться, абонировать}",{абонирам},?,?,abon,,abonovati
28,19625,absorbovati,NaN,v.tr. ipf.,2,absorb,NaN,I,"абсорбировать, поглощать","абсарбаваць, паглынаць",...,"{sorbi, absorbi}","{vstrebávať, absorbovať}","{апсорбира, впива}","{поглощать, абсорбировать}",{абсорбирам},?,?,absorb,,absorbovati
29,24056,abstrahovati,NaN,v.tr. ipf./pf.,1,abstract,NaN,I,абстрагировать,абстрагаваць,...,{abstrakti},{abstrahovať},"{апстрахира, издвојува}",{абстрагировать},{абстрахирам се от},?,?,abstrah,,abstrahovati
43,24070,adaptovati,NaN,v.tr. ipf./pf.,1,adapt,NaN,I,адаптировать,"адаптаваць, прыстасаваць",...,{adapti},"{prispôsobiť, adaptovať}","{прилагодува, адаптира}",{адаптировать},"{адаптирам, пригаждам}",?,?,adapt,,adaptovati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,19473,žužati,NaN,v.intr. ipf.,1,buzz,NaN,NaN,жужжать,"гусці, гудзець",...,"{zumi, zumadi}",{bzučať},{зуи},{жужжать},{жужа},?,?,žuž,,žužati
17913,-37005,råzpečętati,NaN,v.tr. pf.,NaN,unseal,NaN,NaN,распечатать (открыть),распячатаць (адчыніць),...,{!},"{odpečatiť, rozpečatiť}","{распечати, отпечати}",{распечатать (открыть)},"{отпечатам (отворя), !разпечатам}",råz’,pečętati,råzpečęt,,råzpečętati
17914,-37006,råzpečętyvati,NaN,v.tr. ipf.,NaN,unseal,NaN,NaN,распечатывать (открывать),распячатваць (адчыніць),...,"{malŝlosi, elpaki , malfermi}","{odpečaťovať, rozpečaťovať}","{отпечатува, распечатува}",{распечатывать (открывать)},"{отпечатвам (отварям), !разпечатвам}",råz’,pečętyvati,råzpečęt,,råzpečętyvati
17917,-30731,smogti,NaN,v.tr. pf.,NaN,"manage to, be able to, cope, make it",NaN,NaN,"смочь, суметь","змагчы, здолець",...,"{ekpovi, sukcesi}","{zmôcť, zvládnuť}",{!смогне},"{суметь, смочь}",{!смогвам},?,?,smogti,,smogti


In [ ]:
izvoliti -> liti
nezadovoliti -> liti
obvodniti -> dniti
osoliti -> liti
osušiti -> šiti
privoliti -> liti
udariti -> dariti
zadovoliti -> liti

žiti
byti
byvati


råzględěti -> ględěti / ględati
['uniziti'] -> unija



"vo" "so"

True

In [271]:
cand

for aspect in ['ipf.', 'pf.']:
            # print(aspect, len(g[g.partOfSpeech.str.contains(" " + aspect)]))

row.partOfSpeech

'v.tr. pf.'

In [266]:
row

id                                                 17241
isv                                               dariti
addition                                             NaN
partOfSpeech                                   #v.tr.ipf
type                                                   1
en                                                bestow
sameInLanguages                                      NaN
genesis                                              NaN
ru                                                дарить
be                                        дарыць, дорыць
uk                                дарувати, обдаровувати
pl                                     obdarzać, nadawać
cs                     propůjčovat, udělovat, poskytovat
sk                   prepožičiavať, udelovať, poskytovať
bg                             дарявам, одарявам (рядко)
mk                                     дарувам, даруваат
sr                                   даривати, поклањати
hr                    dodijelit

In [263]:
cand

'abdikovati'

In [258]:
bite_all_prefixes_off("izbaviti", "")

'iz’'

In [255]:
KNOWN_BASES = partial_verb_prefixes.query("base_verb != ''").base_verb.unique()

KNOWN_BASES

array(['baviti', 'biti', 'blěskati', 'bodati', 'bryzgati', 'buhati',
       'bzděti', 'cěliti', 'čisliti', 'črpati', 'dariti', 'dniti',
       'ględěti', 'kazati', 'dokonati', 'mysliti', 'pȯlniti', 'rųčiti',
       'dosęgati', 'dostigati', 'dovědati', 'dověriti', 'dovŕšati',
       'drapati', 'drěmati', 'drgati', 'dŕzati', 'dvigati', 'ględnųti',
       'gȯltati', 'gybati', 'hlipati', 'hrapati', 'igrati', 'iskati',
       'izčezati', 'izčrkati', 'izjasniti', 'javiti', 'izključati',
       'kopati', 'koreniti', 'kovati', 'kriknųti', 'kydati', 'mamiti',
       'měnjati', 'iznuriti', 'izobličati', 'izopačati', 'plěti',
       'izpovědati', 'praviti', 'izpråzdniti', 'izprobovati', 'pytati',
       'råbotati', 'rězati', 'rųbati', 'izseliti', 'izslušati',
       'izsmějati', 'izstaviti', 'izstųpati', 'šiti', 'tęgati', 'trgati',
       'izučati', 'izviniti', 'izvlastniti', 'značiti', 'kapati',
       'kupiti', 'kųsati', 'kyhati', 'kymati', 'lajati', 'leskati',
       'lišati', 'liti', 'lizati'

In [ ]:
for partial_verb_prefixes.query("base_verb != ''").base_verb.unique()


In [250]:
g.loc[g.isv.apply(len).idxmin()].isv

'dokonati'

In [248]:
g

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand,base_verb,isv_orig
2071,17286,dokonati,NaN,v.tr. pf.,1,complete,NaN,NaN,"окончить, доделать","скончыць, дарабіць",...,{!kompletaj},"{dorobiť, dokončiť}","{заврши, комплетира}","{доделать, окончить}",{завърша},?,?,dokon,,dokonati
2076,17288,dokonyvati,NaN,v.tr. ipf.,1,complete,NaN,NaN,"оканчивать, завершать, доделывать","сканчаць, завяршаць, дарабляць",...,{!kompletaj},"{dokončovať, dorábať}","{довршува, завршува}","{оканчивать, доделывать, завершать}","{завършвам, запълня}",?,?,dokon,,dokonyvati


In [249]:
stems

2071      konati
2076    konyvati
Name: isv, dtype: object

In [246]:
partial_verb_prefixes.query("isv == @verb_cand")

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand,base_verb,isv_orig
13246,5845,rųčiti,NaN,v.refl. pf.,1,"guarantee, warrant, vouch",NaN,NaN,"поручиться, гарантировать","даручыцца, паручыцца, гарантаваць",...,"{!garantio, mandato, vouch}","{!zaručiť sa, ručiť}","{гарантира, се заземе, ургира}","{поручиться, гарантировать}","{!гаранция за арест, гарантирам}",?,?,rųč,,rųčiti sę


In [241]:
morph.parse(verb_cand)

[Parse(word='rųčiti', tag=OpencorporaTag('Refl,VERB,perf infn'), normal_form='rųčiti', score=1.0, methods_stack=((DictionaryAnalyzer(), 'rųčiti', 180, 0),))]

In [239]:
stems

2188    rųčati
2189    rųčiti
Name: isv, dtype: object

In [226]:
morph.parse(verb_cand)

[Parse(word='rųčiti', tag=OpencorporaTag('Refl,VERB,perf infn'), normal_form='rųčiti', score=1.0, methods_stack=((DictionaryAnalyzer(), 'rųčiti', 180, 0),))]

1693

In [212]:
    stems = g.isv.apply(
        lambda x: x[len(bite_all_prefixes_off(x, "").replace('’', '')):]
    )
    stems

1983     baviti
1984    bavjati
Name: isv, dtype: object

In [214]:
for verb_cand in sorted(stems.values, key=len):
    if verb_cand

['baviti', 'bavjati']

In [209]:
lengths = g.isv.apply(lambda x: bite_all_prefixes_off(x, "")).apply(lambda x: len(x.replace('’', '')))

g.isv.apply(
    lambda x: x[len(bite_all_prefixes_off(x, "").replace('’', '')):]
)


1983     baviti
1984    bavjati
Name: isv, dtype: object

In [202]:
morph.parse("baviti")

[Parse(word='baviti', tag=OpencorporaTag('tran,VERB,impf infn'), normal_form='baviti', score=1.0, methods_stack=((DictionaryAnalyzer(), 'baviti', 73, 0),))]

In [194]:
g.isv.apply(lambda x: bite_all_prefixes_off(x, "")).values.tolist() == ['’', '’']

True

In [196]:
g.isv.apply(lambda x: bite_all_suffixes_off(x, ""))

929     (blěsk, ati+)
933    (blěsk, nųti+)
Name: isv, dtype: object

In [199]:
g.isv.apply(len).idxmin()

929

In [293]:
partial_verb_prefixes.loc[(
    list(partial_verb_prefixes.query("isv.str.startswith('cěniti')").index) +
    list(partial_verb_prefixes.query("isv.str.startswith('ocěn')").index)
)]


,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand,base_verb,isv_orig
1292,24918,cěniti,NaN,v.tr. ipf.,1,appreciate,NaN,NaN,ценить,"шанаваць, цаніць",...,{!ŝati},{ceniť},{цени},{ценить},{ценя},?,?,cěn,,cěniti
8670,14,ocěniti,NaN,v.tr. pf.,1,"appreciate, rate, value, assess, estimate, eva...",NaN,NaN,оценить,"ацаніць, атаксаваць",...,"{rapideco, !estimi, valoro, taksi, takso}","{oceniť, ohodnotiť}",{оцени},{оценить},"{оценка, !оценка, оценка на, стойността}",?,?,ocěn,,ocěniti
8671,144,ocěnjati,NaN,v.tr. ipf.,1,"appreciate, rate, value, assess, estimate, eva...",NaN,NaN,оценивать,"ацэньваць, атаксоўваць, таксаваць",...,"{rapideco, !estimi, valoro, taksi, takso}","{ohodnocovať, oceňovať}",{оценува},{оценивать},"{оценка, !оценка, оценка на, стойността}",?,?,ocěn,,ocěnjati


In [168]:
g

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,de_set,uk_set,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand
8712,29050,odbiti sę,(odbije),v.refl. pf.,1,be reflected,NaN,NaN,"отразиться, отзеркалиться","адбіцца, адлюстравацца",...,{!reflektiert},"{віддзеркалитися, відбитися}",{!reflektos},"{zrkadliť sa, odraziť sa}","{се одрази, се одби}","{отразиться, отзеркалиться}",{!да бъдат отразени},?,?,odb
8713,34100,odbiti sę,(odbije),v.refl. pf.,1,have an effect,NaN,NaN,"отразиться, повлиять, подействовать","адбіцца, паўплываць, зрабіць уплыў, падзейнічаць",...,{!wirken},"{вплинути, подіяти}",{!sian efikon},"{mať vplyv, odraziť sa}",{се одрази},"{подействовать, отразиться, повлиять}",{!има ефект},?,?,odb
8714,20572,odbiti,(odbije),v.tr. pf.,1,"beat off, hit back (ball)",NaN,NaN,отбить (мяч),адбіць (мяч),...,"{schlug zurück (Ball), !schlagen aus}",{відбити (м'яч)},"{!debatos, sukceso reen (pilko)}","{odbiť (lopta), odraziť}","{поврати, одби}",{отбить (мяч)},"{за да отвърне на удара (на топка), !победи}",?,?,odb
8715,20573,odbiti,(odbije),v.tr. pf.,1,repel,NaN,NaN,"отбить, отразить, оттолкнуть","адбіць, адлюстраваць, адпіхнуць, адпхнуць, адш...",...,{!abstoßen},"{відштовхнути, відбити}",{!repeler},"{zahnať, odraziť}","{одби, одврати}","{отразить, оттолкнуть, отбить}",{!удари},?,?,odb
8716,34101,odbiti,(odbije),v.tr. pf.,1,reflect,NaN,NaN,"отразить, отзеркалить","адбіць, адлюстраваць",...,{!reflektieren},"{віддзеркалити, відбити}",{!pripensi},"{reflektovať, odraziť}","{одрази, одби}","{отразить, отзеркалить}",{!отразява},?,?,odb
8717,34103,odbivati sę,NaN,v.refl. ipf.,1,be reflected,NaN,NaN,отражаться,"адбівацца, адлюстроўвацца, люстрыцца, люстравацца",...,{!reflektiert},{відбиватися},{!reflektos},"{zrkadliť sa, odrážať sa}","{се одбива, се рефлектира, се одразува}",{отражаться},{!да бъдат отразени},?,?,odb
8718,34104,odbivati sę,NaN,v.refl. ipf.,1,have an effect,NaN,NaN,"отражаться, влиять, воздействовать","адбівацца, уплываць, рабіць уплыў, паўплываць,",...,{!wirken},{впливати},{!sian efikon},"{mať vplyv, odrážať sa}","{се одразува, има ефект}","{воздействовать, отражаться, влиять}",{!има ефект},?,?,odb
8719,20568,odbivati,NaN,v.tr. ipf.,1,"beat off, hit back (ball)",NaN,NaN,отбивать (мяч),адбіваць (мяч),...,"{schlug zurück (Ball), !schlagen aus}",{відбивати (м'яч)},"{!debatos, sukceso reen (pilko)}","{odbíjať (guľa), odrážať}",{одбива},{отбивать (мяч)},"{за да отвърне на удара (на топка), !победи}",?,?,odb
8720,20570,odbivati,NaN,v.tr. ipf.,1,repel,NaN,NaN,"отбивать, отражать, отталкивать","адбіваць, адлюстроўваць, адштурхоўваць, адпіха...",...,{!abstoßen},"{відштовхувати, відбивати}",{!repeler},"{odrážať, zaháňať}","{одвраќа, одбива}","{отбивать, отталкивать, отражать}",{!удари},?,?,odb
8721,34102,odbivati,NaN,v.tr. ipf.,1,reflect,NaN,NaN,отражать,"адбіваць, адлюстроўваць",...,{!reflektieren},"{віддзеркалювати, відбивати}",{!pripensi},"{reflektovať, odrážať}","{одбива, одразува}",{отражать},{!отразява},?,?,odb


In [163]:
anomalies

[({'ipf.': ['bělěti', 'běliti'], 'pf.': []},
  ['turn white', 'whiten, bleach, blanch'],
  ['белеть', 'белить, отбеливать'],
  ['bieleć', 'bielić, wybielać, zblednąć']),
 ({'ipf.': ['cvětati', 'cvětnųti'], 'pf.': []},
  ['bloom, blossom, flower', 'bloom, blossom, flower'],
  ['цвести', 'цвести'],
  ['kwitnąć', 'kwitnąć']),
 ({'ipf.': ['cěliti', 'cělovati'], 'pf.': []},
  ['aim', 'kiss'],
  ['целиться, прицеливаться', 'целовать'],
  ['celować', 'całować']),
 ({'ipf.': ['davati', 'daviti'], 'pf.': []},
  ['give', 'strangle'],
  ['давать', 'душить, удавливать'],
  ['dawać', 'dusić, dławić']),
 ({'ipf.': ['gasiti', 'gasnųti'], 'pf.': []},
  ['extinguish', 'fade'],
  ['тушить, гасить', 'гаснуть'],
  ['gasić', 'gasnąć']),
 ({'ipf.': ['glåsiti', 'glåsovati'], 'pf.': []},
  ['state, claim, say (text), be worded', 'vote'],
  ['оглашать, утверждать, заявлять, декларировать', 'голосовать'],
  ['głosić, wygłaszać, twierdzić, stwierdzać, oświadczać, brzmieć',
   'głosować']),
 ({'ipf.': ['gnati', '

In [ ]:
 ({'ipf.': ['koristati', 'koristiti'], 'pf.': []},

 ({'ipf.': ['hromati', 'hroměti'], 'pf.': []},
 ({'ipf.': ['cvětati', 'cvětnųti'], 'pf.': []},

 ({'ipf.': ['natęgati', 'natęgnųti'], 'pf.': []},
 ({'ipf.': [], 'pf.': ['odpečętati', 'odpečętyvati']},
 ({'ipf.': ['podstrěkati', 'podstrěknųti'], 'pf.': []},

  

In [157]:
g

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,de_set,uk_set,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem,left_stem_cand
9706,18169,osvěžati,NaN,v.tr. ipf.,1,"refresh, freshen",NaN,NaN,освежать,асвяжаць,...,"{auffrischen, !Refresh}","{робити свіжим, освіжати}","{!refreŝigi, refreŝigi}","{osviežovať, občerstvovať}",{освежува},{освежать},"{освежаване, !обновяване}",?,?,osvěž
9707,18171,osvěžiti,NaN,v.tr. pf.,1,"refresh, freshen",NaN,NaN,освежить,"асвяжыць, адсвяжыць",...,"{auffrischen, !Refresh}","{освіжити, зробити свіжим}","{!refreŝigi, refreŝigi}","{osviežiť, občerstviť}",{освежи},{освежить},"{освежаване, !обновяване}",?,?,osvěž


(1, 46)
(1, 46)


In [113]:
'privęzyvati'.endswith('yvati')

True

In [94]:
bite_all_prefixes_off('nedoråzuměti', '')

'ne’do’råz’u’'

In [97]:
vc = partial_verb_prefixes.isv.apply(lambda x: bite_all_prefixes_off(x, '')).value_counts()

In [103]:
partial_verb_prefixes.query('isv.str.startswith("priv")')

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,en_set,de_set,uk_set,eo_set,sk_set,mk_set,ru_set,bg_set,prefix,verb_stem
11890,23180,privabiti,NaN,v.tr. pf.,2,attract,uk z sl,NaN,привлечь,"прыцягнуць, завалаць, прывабіць, залучыць",...,{attract},{!anlocken},"{привабити, принадити}",{!altiri},"{prilákať, !privábiť}",{привлече},{привлечь},{!привличане на},?,?
11891,23182,privabjati,NaN,v.tr. ipf.,2,attract,uk z sl,NaN,привлекать,"прыцягваць, прывабліваць, вабіць",...,{attract},{!anlocken},"{принаджувати, приваблювати}",{!altiri},"{lákať, !vábiť}",{привлекува},{привлекать},{!привличане на},?,?
11894,31950,privesti,(privede),v.tr. pf.,1,"bring, adduce",NaN,NaN,"привести, доставить","прывесці, давесці, напрыводзіць, падаць, даста...",...,"{adduce, bring}","{adduce, !bringen}","{доставити, привести}","{!venigu, pretekstas}","{uviesť, !priviesť}","{донесе, принесе}","{привести, доставить}",{!донесе доведе},?,?
11895,31952,privesti,(privede),v.tr. pf.,1,"bring about, evoke, lead to",NaN,NaN,"привести (к последствиям), вызвать, спровоциро...","прывесці, давесці (да наступстваў), выклікаць,...",...,"{evoke, lead to, bring about}","{!bewirken, hervorrufen, führen zu}","{викликати, спровокувати, призвести (до наслід...","{!provoki, elvoki, kondukas al}","{zapríčiniť, vyvolať, spôsobiť, viesť k, !priv...","{приведе, доведе}","{вызвать, спровоцировать, привести (к последст...","{да доведе до, да предизвика, !да}",?,?
11899,21482,privęzati,(privęže),v.tr. pf.,1,tie (to smth.),NaN,NaN,привязать,"прывязаць, прыпяць, прыпнуць",...,{tie (to smth.)},{!Binder (etw.)},{прив'язати},{!ligi (al smth.)},"{!priviazať, uviazať}",{приврзе},{привязать},{!вратовръзка (за нещо л.)},?,?
11900,3471,privezti,NaN,v.tr. pf.,1,bring,NaN,NaN,привезти,прывезці,...,{bring},{!bringen},{привезти},{!venigu},{!priviezť},"{донесе, принесе}",{привезти},{!донесе},?,?
11901,21483,privęzyvati,NaN,v.tr. ipf.,1,tie (to smth.),NaN,NaN,привязывать,прывязваць,...,{tie (to smth.)},{!Binder (etw.)},{прив'язувати},{!ligi (al smth.)},"{uväzovať, !priväzovať}",{приврзува},{привязывать},{!вратовръзка (за нещо л.)},?,?
11909,1965,privitati,NaN,v.tr. pf.,2,welcome,v z,NaN,"приветствовать, встречать","прывітаць, вітаць, сустракаць, спатыкаць, стра...",...,{welcome},{!herzlich willkommen},{вітати},{!bonvenon},"{!privítať, uvítať}","{посака добредојде, поздрави}","{приветствовать, встречать}",{!добре дошли},?,?
11910,23175,privlåčivati,NaN,v.tr. ipf.,2,attract,ru j,NaN,"привлекать, притягивать","прыцягваць, прыцягаць, прывабліваць, вабіць",...,{attract},{!anlocken},"{притягувати, притягати}",{!altiri},{!priťahovať},{привлекува},"{притягивать, привлекать}",{!привличане на},?,?
11911,22501,privlastniti sobě,NaN,v.tr. pf.,2,"appropriate, arrogate",z,NaN,присвоить,"прыўлашчыць, прыўласціць, прысвоіць, прысабечы...",...,"{arrogate, appropriate}",{!falls anmaßen},{присвоїти},"{arrogate, !taŭga}","{!privlastniť si, prisvojiť si, prisúdiť si}",{присвои},{присвоить},"{определят, !подходящо}",?,?


In [100]:
vc[vc > 10]

’         1065
råz’       272
s’         265
iz’        231
o’         217
od’        204
za’        188
ob’        167
u’         167
po’        161
na’        157
v’         132
pri’        90
prě’        82
pro’        79
do’         67
pod’        67
o’s’        58
na’s’       52
od’s’       40
po’s’       39
vȯz’        39
u’s’        35
råz’s’      34
za’s’       32
prě’s’      30
ob’s’       27
iz’s’       27
råz’v’      26
s’v’        23
do’s’       22
od’v’       22
u’v’        22
sȯ’         20
iz’v’       19
po’v’       19
s’po’       17
pri’s’      17
pro’s’      17
nad’        17
prěd’       16
na’v’       16
o’v’        15
ob’v’       14
de’         14
za’v’       14
o’s’v’      13
pri’v’      13
do’v’       11
Name: isv, dtype: int64

In [93]:
for word in partial_verb_prefixes[partial_verb_prefixes.isv.str[-5:] == 'uměti'].isv.values:
    prefs = bite_all_prefixes_off(word, 'uměti')
    print(word, morphemes_split(word), prefs)
    

nedoråzuměti {('', 'nedoråzumě', 'ti')} ne’do’råz’
obezuměti {('', 'obezumě', 'ti')} obez’
råzuměti {('', 'råzumě', 'ti')} råz’
sråzuměti {('', 'sråzumě', 'ti')} s’råz’
šuměti {('', 'šum', 'ěti')} ’
uměti {('', 'umě', 'ti')} ’


In [75]:

bite_all_prefixes_off('šuměti', 'uměti')

[]

In [73]:
'nedoråzuměti'.endswith('uměti')

True

In [59]:
for word in partial_verb_prefixes[partial_verb_prefixes.isv.str[-4:] == 'lati'].isv.values:
    print(word, morphemes_split(word))

blågoželati {('', 'blågožela', 'ti')}
dělati {('', 'děla', 'ti')}
izdělati {('', 'izděla', 'ti')}
izslati {('', 'izsl', 'ati')}
izsylati {('', 'izsyla', 'ti')}
mglati {('', 'mgla', 'ti')}
nadělati {('', 'naděla', 'ti')}
nakalati {('', 'nakala', 'ti')}
odslati {('', 'odsl', 'ati')}
odsylati {('', 'odsyla', 'ti')}
osedlati {('', 'osedla', 'ti')}
poblågoželati {('', 'poblågožela', 'ti')}
poslati {('', 'posl', 'ati')}
postlati {('', 'post', 'lati')}
posylati {('', 'posyla', 'ti')}
poželati {('', 'požela', 'ti')}
pridělati {('', 'priděla', 'ti')}
prislati {('', 'prisl', 'ati')}
prisylati {('', 'prisyla', 'ti')}
råzkalati {('', 'råzkala', 'ti')}
råzsedlati {('', 'råzsedla', 'ti')}
råzslati {('', 'råzsl', 'ati')}
råzstlati {('', 'råzst', 'lati')}
råzsylati {('', 'råzsyla', 'ti')}
sdělati {('', 'sděla', 'ti')}
sedlati {('', 'sedla', 'ti')}
slati {('', 'sl', 'ati')}
stlati {('', 'st', 'lati')}
zamglati {('', 'zamgla', 'ti')}
želati {('', 'žela', 'ti')}


In [58]:
for word in partial_verb_prefixes[partial_verb_prefixes.isv.str[-5:] == 'eliti'].isv.values:
    print(word, morphemes_split(word))

izpepeliti {('', 'izpepel', 'iti')}
izseliti {('', 'izsel', 'iti')}
naseliti {('', 'nasel', 'iti')}
posteliti {('', 'postel', 'iti')}
prěseliti {('', 'prěsel', 'iti')}
råzsteliti {('', 'råzstel', 'iti')}
råzveseliti {('', 'råzvesel', 'iti')}
steliti {('', 'stel', 'iti')}
veseliti {('', 'vesel', 'iti')}


In [64]:
morph.word_is_known("vesel")

True

In [65]:
morph.parse("vesel")

[Parse(word='vesel', tag=OpencorporaTag('neut,NOUN gent,plur'), normal_form='veslo', score=1.0, methods_stack=((DictionaryAnalyzer(), 'vesel', 129, 5),))]

In [50]:
morph.parse("uměti")

[Parse(word='uměti', tag=OpencorporaTag('auxv,VERB,impf infn'), normal_form='uměti', score=1.0, methods_stack=((DictionaryAnalyzer(), 'uměti', 103, 0),))]

In [48]:
len(_ISV_PREFFIXES)

27

In [ ]:
'ovati' 'ovyvati'
'lati', 'eliti'

In [43]:
for cand_set in candidates[(2, 6)]:
    print("========")
    for word in cand_set[0]:
        parsings = morph.parse(word)
        print(word)
        for parse in parsings:
            print(extract_stem_prefix_suffix(word, parse, isv_dict), parse.tag)


pakovati
('', 'pak', 'ovati') tran,VERB,impf infn
spakovati
('', 'spak', 'ovati') tran,VERB,perf infn
('', 'spak', 'ovati') Refl,VERB,perf infn
upakovati
('', 'upak', 'ovati') tran,VERB,perf infn
upakovyvati
('', 'upakovyva', 'ti') tran,VERB,impf infn
zapakovati
('', 'zapak', 'ovati') tran,VERB,perf infn
zapakovyvati
('', 'zapakovyva', 'ti') tran,VERB,impf infn
posteliti
('', 'postel', 'iti') tran,VERB,perf infn
postlati
('', 'post', 'lati') tran,VERB,perf infn
råzsteliti
('', 'råzstel', 'iti') tran,VERB,perf infn
råzstlati
('', 'råzst', 'lati') tran,VERB,perf infn
steliti
('', 'stel', 'iti') tran,VERB,impf infn
stlati
('', 'st', 'lati') tran,VERB,impf infn


In [41]:
parse

Parse(word='pakovati', tag=OpencorporaTag('tran,VERB,impf infn'), normal_form='pakovati', score=1.0, methods_stack=((DictionaryAnalyzer(), 'pakovati', 41, 0),))

In [22]:
en_uniq

'complicate'

In [ ]:
slovnik.query("partOfSpeech.str.contains('v.')")

In [ ]:
_ISV_PREFFIXES = [
    "do", "iz", "na", "nad", "nedo", "o", "ob", "ob", "obez", "od", "po", "pod", "prě", "prěd", "pri", "pro", 
    "raz", "de", "s", "so", "su", "u", "v", "vo", "voz", "vy", "za",
]
